In [ ]:
from petastorm import make_reader
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kurtosis,skew,pearsonr
from scipy.signal import periodogram,correlate,correlation_lags
import scipy.signal
from ecgdetectors import Detectors
import matplotlib.ticker as ticker
import pywt
import neurokit2 as nk
import biosppy
from matplotlib.widgets import TextBox, Button
import sys
import pandas as pd
import os
sys.path.append(os.path.join(os.getcwd(), ".."))
from shared_utils import Time_series_dimensions_calculus as TSD


In [ ]:
##Read the dataset
path_formatted_glasgow = "/workspaces/maitrise/data/20220902_data_physio_formatted_merged/merged/dataParquet"
path_petastorm = f"file:///{path_formatted_glasgow}"
with make_reader(path_petastorm) as reader:
    for sample in reader:
        data = sample
        break


In [ ]:
print(data)

In [ ]:
def plot_ECG_signal(signal,name,length= data.signal_length,fs = data.sampling_frequency):
     x = np.array(range(0,(len(signal))))
     x = x/fs    
     fig,ax = plt.subplots(nrows = 1,ncols = 2, figsize = (20,10))
     ax[0].plot(x,signal)
     ax[0].set_title(f"Full signal of Lead {name.decode('utf8')}")
     ax[0].grid()
     ax[1].plot(x,signal)
     ax[1].set_title(f"Close up signal of Lead {name.decode('utf8')}")
     ax[1].grid()
     if len(x) == data.signal_length:
          ax[1].set_xlim([0,3])
     else :
          ax[1].set_xlim([0,x[-1]])
     plt.show()

ECG_signal = data.signal
ECG_lead = data.signal_names
fs = data.sampling_frequency
dico_ECG = {}

for i,j in zip(ECG_lead,range(12)):
     dico_ECG[i] = ECG_signal[:,j]
     print(dico_ECG[i].shape)
     #plot_ECG_signal(dico_ECG[i],i)
     



In [ ]:
##Kurtosis SQI factor :
def kurtosis_SQI(signal):
    return kurtosis(signal)

##Skewness SQI factor:
def Skewness_SQI(signal):
    return skew(signal)

# for i in ECG_lead:
#     print(f"for lead {i.decode('utf8')} : ")
#     for j in range(5):
#         print("Kurtosis = ",kurtosis_SQI(dico_ECG[i][j*1000:(1000)*(j+1)]), " and Skewness = ", Skewness_SQI(dico_ECG[i][j*1000:(1000)*(j+1)]))
#     print("For the entire signal : Kurtosis = ",kurtosis_SQI(dico_ECG[i])," and Skewness = ",Skewness_SQI(dico_ECG[i]))
#     print("For the entire signal : Comb_SQI = ",(np.abs(Skewness_SQI(dico_ECG[i]))/5)*kurtosis_SQI(dico_ECG[i]))
#     fig,ax = plt.subplots(nrows = 1,ncols = 2, figsize = (20,10))
#     ax[0].plot(dico_ECG[i])
#     ax[0].grid()
#     ax[0].set_title(f"lead {i.decode('utf8')}")
#     ax[1].hist(dico_ECG[i])
#     ax[1].set_title(f"Histogram for lead {i.decode('utf8')}")
#     plt.show()

In [ ]:
##Kurtosis and Skewness for 100 interval length through the signal

def Kurtosis_signal(signal,fs,length = 100):
    X = np.c_[[signal[int((w - 1)) : int((w) + length)] for w in range(1, int(len(signal) - length))]]
    K_arr = np.array([kurtosis(X[i,:]) for i in range(X.shape[0])])
    return K_arr,np.array([range(1, int(len(signal) - length))])

def Skewness_signal(signal,fs,length = 100):
    X = np.c_[[signal[int((w - 1)) : int((w) + length)] for w in range(1, int(len(signal) - length))]]
    S_arr = np.array([Skewness_SQI(X[i,:]) for i in range(X.shape[0])])
    return S_arr,np.array([range(1, int(len(signal) - length))])

for i in ECG_lead:
    #plt.figure()
    #fig,ax = plt.subplots(nrows = 3,ncols=1,figsize = (15,15))
    K,inter = Kurtosis_signal(dico_ECG[i],fs,500)
    print(f"Mean Kurtosis for Lead {i.decode('utf8')} : ",np.mean(K))
    #S,_ = Skewness_signal(dico_ECG[i],fs,500)
    # fig.tight_layout(h_pad=4)
    # ax[0].plot(inter[0,:]/fs,K)
    # ax[0].set_xlabel("Number of segment")
    # ax[0].set_ylabel("Kurtosis")
    # ax[0].set_title("Kurtosis Time evolution")
    # ax[0].grid()
    # ax[1].plot(inter[0,:]/fs,S)
    # ax[1].set_xlabel("Number of segment")
    # ax[1].set_ylabel("Skewness")
    # ax[1].set_title("Skewness Time evolution")
    # ax[1].grid()
    # ax[2].plot(np.linspace(0,int(len(dico_ECG[i])/fs),len(dico_ECG[i])),dico_ECG[i])
    # ax[2].set_xlabel("Time Interval")
    # ax[2].set_ylabel("Amplitude Voltage")
    # ax[2].set_title(f"Lead {i.decode('utf8')}")
    # ax[2].grid()
    # plt.show()

    



In [ ]:
for i in ECG_lead : 
    print(f"Kurtosis value for Lead {i.decode('utf8')} : ", kurtosis_SQI(dico_ECG[i]))

In [ ]:
#SDR factor :

def SDR(signals,ECG_lead,fs = data.sampling_frequency,sign_length = data.signal_length):
    x = np.array(range(0,int(sign_length)))
    x = x/fs
    for i in ECG_lead:
        # for j in range(int(data.signal_length/1000)):
        #     f,PSD = periodogram(signals[i][j*1000:(1000)*(j+1)],fs)
        #     ECG_signal_PSD = np.sum(PSD[np.logical_and(f>5,f<14)])
        #     ECG_tot = np.sum(PSD[np.logical_and(f>5,f<50)])
        #     SDR_val = ECG_signal_PSD/ECG_tot
        #     print(f"for lead {i.decode('utf8')} between [{j*1000},{(j+1)*(1000)}]:",SDR_val)
        
        f,PSD = periodogram(signals[i],fs)
        QRS_signal_PSD = np.sum(PSD[np.logical_and(f>5,f<14)])
        ECG_tot = np.sum(PSD[np.logical_and(f>5,f<50)])
        SDR_val = QRS_signal_PSD/ECG_tot
        XECG = scipy.fft.fft(dico_ECG[i])
        XECG = (2/data.signal_length)*XECG[:data.signal_length//2]
        xf = scipy.fft.fftfreq(data.signal_length,1/data.sampling_frequency)[:data.signal_length//2]
        IOR = np.sum(np.abs(XECG[np.logical_and(xf>5,xf<40)]))/(np.sum(np.abs(XECG[np.logical_and(xf>0,xf<120)]))-np.sum(np.abs(XECG[np.logical_and(xf>5,xf<40)])))
        print(f"SDR for the entire signal of lead {i.decode('utf8')} :",SDR_val)
        print(f"IOR for the entire signal of lead {i.decode('utf8')} :",IOR)
        fig,ax = plt.subplots(nrows = 2,ncols = 2, figsize = (20,20))
        ax[0,0].plot(x,dico_ECG[i])
        ax[0,0].set_title(f"Entire signal for lead {i.decode('utf8')}")
        ax[0,0].grid()
        ax[0,1].plot(x,dico_ECG[i])
        ax[0,1].set_title(f"Close up signal for lead {i.decode('utf8')}")
        ax[0,1].set_xlim([0,2])
        ax[0,1].grid()
        ax[1,0].plot(xf,np.abs(XECG))
        ax[1,0].set_title(f"Magnitude of the FT of lead {i.decode('utf8')}")
        ax[1,0].set_xlim([0,120])
        ax[1,0].grid()
        ax[1,1].semilogy(f, PSD)
        ax[1,1].set_ylim([1e-7, 1e2])
        ax[1,1].set_xlim([0,120])
        ax[1,1].set_title(f"PSD of lead {i.decode('utf8')}")
        ax[1,1].grid()
        plt.show()
        #plt.semilogy(f, PSD)
        #plt.ylim([1e-7, 1e2])
        #plt.xlabel('frequency [Hz]')
        #plt.ylabel('PSD [V**2/Hz]')
        #plt.show()
        
SDR(dico_ECG,ECG_lead)       

In [ ]:
#Beats detector ECG
def QRS_detector_and_extractor(signal,lead,fs = data.sampling_frequency,sign_len = data.signal_length):
    beats_extractor = {}
    range_lead_beats = {}
    detect = Detectors(fs)
    x = np.array(range(0,int(sign_len)))
    x = x/fs
    for i in lead:
        r_peaks = detect.pan_tompkins_detector(signal[i])
        plt.figure()
        fig,ax  = plt.subplots(nrows = 1,ncols=2,figsize=(15,15))
        ax[0].plot(x,signal[i],marker = "o",markerfacecolor = "red",color = "b",markevery = r_peaks,label = "R-peaks detected")
        ax[0].legend()
        ax[0].set_title(f"R-peaks for the entire signal of lead {i.decode('utf-8')}")
        ax[1].plot(x,signal[i],marker = "o",markerfacecolor = "red",color = "b",markevery = r_peaks,label = "R-peaks detected")
        ax[1].legend()
        ax[1].set_title(f"R-peaks for a small signal interval  of lead {i.decode('utf-8')}")
        ax[1].set_xlim([0,1.5])

        ##We don't want first and last beat:
        r_peaks = r_peaks[2:-2]
        lead_beats_extracted = {}
        range_peaks = np.array([])
        for peaks in r_peaks:
            ##Extract each bets:
            range_peaks = np.append(range_peaks,f"{peaks}")
            if 400+peaks <= sign_len:
                lead_beats_extracted[str(peaks)] = signal[i][(np.abs(250-peaks)):(400+peaks)]
            elif 250-peaks <0:
                lead_beats_extracted[str(peaks)] = signal[i][0:(400+peaks)]
            else : 
                lead_beats_extracted[str(peaks)] = signal[i][(np.abs(250-peaks)):-1]
        beats_extractor[i] = lead_beats_extracted
        range_lead_beats[i] = range_peaks
        plt.figure()
        fig,ax  = plt.subplots(nrows = 1,ncols=2,figsize=(15,15))
        ax[0].plot(x,signal[i])
        ax[0].set_title(f"R-peaks for the entire signal of lead {i.decode('utf-8')}")
        ax[1].plot(x[(np.abs(250-int(range_peaks[0]))):(400+int(range_peaks[0]))],lead_beats_extracted[range_peaks[0]])
        ax[1].set_title(f"R-peaks for a small signal interval  of lead {i.decode('utf-8')}")
    return beats_extractor,range_lead_beats
class_beats,range_beats = QRS_detector_and_extractor(dico_ECG,ECG_lead)

In [ ]:
print(class_beats)
print(range_beats)


In [ ]:
##Wavelet decomposition

for i in ECG_lead:
    fig, ax = plt.subplots(figsize=(6,1))
    ax.set_title(f"Original signal for lead {i.decode('utf8')}")
    ax.plot(dico_ECG[i])
    plt.show() 
    data1 = dico_ECG[i]
    waveletname = 'db4'
 
    fig, axarr = plt.subplots(nrows=10, ncols=2, figsize=(10,10))
    for ii in range(10):
        (data1, coeff_d) = pywt.dwt(data, waveletname)
        axarr[ii, 0].plot(data1, 'r')
        axarr[ii, 1].plot(coeff_d, 'g')
        axarr[ii, 0].set_ylabel("Level {}".format(ii + 1), fontsize=14, rotation=90)
        axarr[ii, 0].set_yticklabels([])
        if ii == 0:
            axarr[ii, 0].set_title("Approximation coefficients", fontsize=14)
            axarr[ii, 1].set_title("Detail coefficients", fontsize=14)
        axarr[ii, 1].set_yticklabels([])
    plt.tight_layout()
    plt.show()  


In [ ]:
### Try with another python method:
waveletname = 'db4'
level_w = 9
i = ECG_lead[1]

def Wavelet_coef(sig,name,lev):
    All_coeff = pywt.wavedec(sig,name,level = lev)

    CA_w = All_coeff[0]
    CD_w = All_coeff[1:len(All_coeff)]
    #fig, ax = plt.subplots(figsize=(6,1))
    #ax.set_title(f"approximation signal at level {lev} for lead {i.decode('utf8')}")
    #ax.plot(CA_w)
    #plt.show()

    #fig, axarr = plt.subplots(nrows=lev, ncols=1, figsize=(lev,lev))
    #for j in range(len(CD_w)):
        #axarr[j].plot(np.asarray(CD_w)[-(j+1)])
        #axarr[j].set_ylabel("Level {}".format(j + 1), fontsize=14, rotation=90)
        #axarr[j].set_yticklabels([])
        #if j ==0:
            #axarr[j].set_title("detail coefficients", fontsize=14)
    
    #plt.tight_layout()
    #plt.show()
    return CA_w,CD_w  


In [ ]:
##Calcuation of the wPMF (wavelet probability  mass function):

def Energy_L2(coeff):
    return np.sum(np.abs(coeff)**2)


x = np.array(range(len(dico_ECG[ECG_lead[0]])))
x = x/data.sampling_frequency
for i in ECG_lead:
    CA_w,CD_w = Wavelet_coef(dico_ECG[i],waveletname,level_w)
    p = np.array([])
    for CD in range(level_w):
        p = np.append(p,Energy_L2(np.asarray(CD_w)[-(CD+1)]))
    p = np.append(p,Energy_L2(np.asarray(CA_w)[0]))
    Etot = np.sum(p)
    p = p/Etot
    SQI_ECG = np.sum(p[3:6])
    print(f"SQI_QRS for lead {i.decode('utf8')} :",SQI_ECG)
    print("Check that 'sum of all P = 1' :",np.sum(p))
    fig,ax = plt.subplots(nrows=3, ncols=1, figsize=(15,10))
    ax[0].set_title(f"Original signal from lead {i.decode('utf8')} ")
    ax[0].plot(x,dico_ECG[i],'r')
    ax[0].grid()
    ax[1].set_title(f" Closed up look of signal from lead {i.decode('utf8')} ")
    ax[1].plot(x,dico_ECG[i],'b')
    ax[1].set_xlim([0,3])
    ax[1].grid()
    ax[2].set_title(f"wPMF for each detail coefficient level of lead {i.decode('utf8')} ")
    ax[2].bar(range(1,level_w+2),p)
    plt.xticks(range(1,level_w+2),range(1,level_w+2))
    ax[2].set_xlabel("Level")
    ax[2].set_ylabel("wPMF")
    plt.show()

    
    


In [ ]:
##MSSR method
def conv_m(a, b, mode='full'):
    """Convolve a vector with collection of vectors.
    Convolve a 1D array `a` with each column of the 2D array `b`.   
    
    Convolution is carried out with `scipy.signal.fftconvolve`
    
    Parameters
    ----------
    a : 1D array
        1D array input
        
    b : 1D or 2D array_like
        1D or 2D array input
        
    mode : str {'full', 'same'}, optional
        A string indicating the size of the output:
        ``full``
           The output is the full discrete linear convolution
           of the inputs. (Default)
        ``same``
           The output is the same size as `a`, centered
           with respect to the 'full' output.
    Returns
    -------
    c : 2D array
        A 2D array where each columns corresponds to the 
        convolution of `a` and a column of `b`
    
    See Also
    --------
    `scipy.signal.fftconvolve()`
        
    """
    # input vector 'a' to 1 dimension
    a = a.ravel()
    # number of samples vector 'a'
    siz_a = len(a)
    
    # input 'b' as 2D matrix [samples, columns]
    try:
        b.shape[1]
    except IndexError:
        b = b[:, np.newaxis]
    
    # number of samples and number of channels in input 'b'
    siz_b, col_b = b.shape
    
    # allocate space for result
    if mode == 'same':
        c = np.zeros((siz_a, col_b) , dtype = complex)
    elif mode == 'full':
        N = siz_a + siz_b - 1
        c = np.zeros((N , col_b), dtype = complex)
    
    # 1D convolutions per columns in 'b'    
    for ix in range(0 , col_b):
         c[:,ix] = scipy.signal.fftconvolve(a, b[:,ix] , mode)
         
    return c


def epoching(data, samples_epoch, samples_overlap = 0):
    """Divide an array in a colletion of smaller arrays
    
    Divides the `data` provided as [n_samples, n_channels] using the 
    `size_epoch` indicated (in samples) and the `overlap_epoch` between 
    consecutive epochs.
   
    Parameters
    ----------
    data : 2D array 
        with shape (n_samples, n_channels)
    samples_epochs : 
        number of samples in smaller epochs
        
    samples_overlap : 
        number of samples for ovelap between epochs (Default 0)
    Returns
    -------
    epochs : 3D array 
        with shape (samples_epoch, n_channels, n_epochs)
    
    remainder : 2D array 
        with the remaining data after last complete epoch
    
    ix_center : 1D array
        indicates the index tha corresponds to the center of the nth epoch.
    """ 
    # input 'data' as 2D matrix [samples, columns]
    try:
        data.shape[1]
    except IndexError:
        data = data[:, np.newaxis]
    
    # number of samples and number of channels
    n_samples, n_channels = data.shape

    # Size of half epoch
    half_epoch = np.ceil(samples_epoch / 2 )

    # Epoch shift   
    samples_shift = samples_epoch - samples_overlap

    # Number of epochs
    n_epochs =  int(np.floor( (n_samples - samples_epoch) / float(samples_shift) ) + 1 )
    if n_epochs == 0:
        return np.array([]), data, np.array([])

    #markers indicates where the epoch starts, and the epoch contains samples_epoch rows
    markers = np.asarray(range(0,n_epochs)) * samples_shift
    markers = markers.astype(int)

    #Divide data in epochs
    epochs = np.zeros((samples_epoch, n_channels, n_epochs))
    ix_center = np.zeros((n_epochs,1))

    for i_epoch in range(0,n_epochs):
        epochs[:,:,i_epoch] = data[ markers[i_epoch] : markers[i_epoch] + samples_epoch ,:]
        ix_center[i_epoch] = markers[i_epoch] -1 + half_epoch
        
    if ( (markers[-1] + samples_epoch) < n_samples): 
        remainder = data[markers[-1] + samples_epoch : n_samples, :]
    else:
        remainder = np.asarray([])
    
    return epochs, remainder, ix_center.astype(int)

def iepoching(epochs, shift_epoch):
    """
    Merges a set of epochs [n_samples_epoch, n_channels] into  
    the complete signal(s) x [n_samples, n_channels] taking into account
    the shift between consecutive epochs
   
    Parameters
    ----------
    epochs : 2D array_like with shape (n_samples, n_channels)
    shift_epoch : number of samples in smaller epochs
    Returns
    -------
    x : 2D array with shape (samples_epoch, n_channels, n_epochs)
    """
    
    # obtain parameters
    (size_epoch, n_channels, n_epochs) = epochs.shape
    n_samples = (shift_epoch * (n_epochs - 1)) + size_epoch
    ix = np.arange(n_epochs) * shift_epoch
    
    # merging matrix
    merging = np.zeros((n_samples, n_channels, 2))
    # Number of epochs that contribute for a specific point
    n_merging = np.zeros((n_samples, n_channels, 2))
    
    for i_epoch in range(n_epochs):
        merging[ix[i_epoch] : ix[i_epoch] + size_epoch, :, 1 ] = epochs[:, :, i_epoch]
        n_merging[ix[i_epoch] : ix[i_epoch] + size_epoch, :, 1] = 1
        merging[:,:,0] = np.sum(merging, axis=2)
        n_merging[:,:,0] = np.sum(n_merging, axis=2)
        merging[ix[i_epoch] : ix[i_epoch] + size_epoch, :, 1 ] = 0
        n_merging[ix[i_epoch] : ix[i_epoch] + size_epoch, :, 1 ] = 0
    
    x = np.divide(merging[:,:,0], n_merging[:,:,0])    
    return x 


def cmorlet_wavelet(x, fs, freq_vct, n=6, normalization=True):
    """Perform the continuous wavelet (CWT) tranform using the complex Morlet wavelet.
    
    Parameters
    ----------
    x  : 1D array with shape (n_samples) or 
         2D array with shape (n_samples, n_channels)
    fs : Sampling frequency 
        in Hz
    freq_vct : 1D array 
        with frequencies to compute the CWT (Default = [1 : 1 : fs/2] )
    n : Number of cicles inside the Gaussian curve 
        (Default 6)
    normalization : Scale each wavelet to have energy equal to 1
        (Default True) 
    Returns
    -------
    wcoef : Complex wavelet coefficients 
            2D array with shape [n_samples, n_freqs] if `x` is 1D array
            3D array with shape [n_samples, n_freqs, n_channels] if `x` is 2D array
    
    wfam  : 2D array with shape [n_wavelet_samples, n_freqs] where each column
            corresponds to the a member of the wavelet family
    
    """
    # input 'x' as 2D matrix [samples, columns]
    try:
        x.shape[1]
    except IndexError:
        x = x[:, np.newaxis]
    
    # number of samples and number of channels
    n_samples, n_channels = x.shape
    
    # number of wavelets
    n_freqs = len(freq_vct)

    # number of samples for Wavetet family
    # This is equal to the number of samples needed to represent 2*n cycles 
    # of a sine with frequency = fres(1)[Hz], sampled at fs [Hz]. 
    # This is done to ensure that every wavelet in the wavalet family will be 
    # close to 0 in the negative and positive edges
    n_samples_wav = np.round( (2*n/freq_vct[0])*fs )

    # The wavelet will be symmetrical around 0
    if np.mod(n_samples_wav,2) == 0: # even samples
        n_samples_wav = n_samples_wav + 1

    # create time vector for Wavelet family
    half = np.floor(n_samples_wav/2)
    time = np.arange(-half, half+1)/fs

    # initialize Wavelet family matrix
    wfam = np.zeros([len(time), n_freqs], dtype=complex)

    # for each frequency defined in FREQ, create its respective Wavelet
    for iwav in range(n_freqs):
        s = n/(2*np.pi*freq_vct[iwav])
        gaussian_win = np.exp((-time**2)/(2*s**2))
        sinwave = np.exp(2*np.pi*1j*freq_vct[iwav]*time)
        if normalization:
            # each wavelet has unit energy sum(abs(wavelet).^2)) = 1
            A = 1. / ((s**2) * np.pi) ** (1./4)
        else:
            A = 1.
        # Complex Morlet wavelet
        wfam[:, iwav] = A * sinwave * gaussian_win

    wcoef = np.zeros((n_samples, n_freqs, n_channels), dtype=complex)

    if n_channels == 1:
        # one channel
        tmp = conv_m(x, wfam, 'same')
        wcoef[:, :, 0] = tmp        
    else:
        # convolution between signal X and the each Wavelt in the Wavelet family
        for i_channel in range(n_channels):
            x_tmp = x[:, i_channel]
            tmp = conv_m(x_tmp, wfam, 'same')
            wcoef[:, :, i_channel] = tmp     

    return wcoef, wfam


def rfft(x, n=None, dim=None):
    """Real Fast Fourier Transform.
    
    Considering a real signal A with B = fft(A), B is Hermitian symmetric,
    i.e. B(-1) = conj(B(1)), therefore the complete spectrum B
    can be found by using with only the non-negative frequencies in B
    
    
    Parameters
    ----------
    x : 1D array with shape (n_samples) or
        2D array with shape (n_samples, n_channels)
    
    n : Number of samples to compute the FFT
        (Default = n_samples in array x)    
    dim : Dimension to compute the RFFT 
        (Default:  first array dimension whose size does not equal 1)
    Returns
    -------
    y  : Non-negative complex spectrum of `x` with shape as `x`
    
    See Also
    --------
    `np.fft.fft()`
    
    """

    # shape of x
    shape_x = x.shape
    # number of dimentions
    dim_x = len(shape_x)
    
    # limits to 2-dimention data
    assert dim_x<=2
    
    # check shape of X, and set n and dim defaults
    if dim_x == 1:
        dim_def = 0
    else:
        if shape_x[0] == 1:
            # shape [1, n_samples] (row vector)
            dim_def = 1
        elif shape_x[1] == 1:
            # shape [n_samples, 1] (column vector)
            dim_def = 0 
        else:
            # X is a 2D Matrix, a shape [n_samples, n_channels] is asummed
            dim_def = 0
    
    if dim is None:
        dim = dim_def
    
    if n is None:
        n = shape_x[dim]
    
    # FFT
    yc = np.fft.fft(x, n=n, axis=dim)
    
    # points to keep
    if n%2 == 0:    
        # even case
        n_new = int((n / 2) + 1)
    else:
        # odd case
        n_new = int((n + 1) / 2)
    
    if dim_x == 1:
        y = yc[0:n_new]
    else:
        if dim == 0:
            y = yc[0:n_new,:]        
        else:
            y = yc[:, 0:n_new]
    
    return y

def irfft(y, n=None, dim=None):
    '''
    The IRFFT function returns the Inverse DFT (using the RFFT algorithm)of
    a spectrum Y containing ONLY the positive frequencies, with the
    assumption than Y is the positive half of a Hermitian Symmetric spectrum
    from a real signal X.
        
    Parameters
    ----------
    y : 1D or 2D array with the positive spectrum of 
        real-valued signals with shape (n_samples, n_channels)
    n : Number of samples in the original x signals 
        N not provided. Y is assumed be obtained from a signal X with even number fo samples 
    dim : Dimension to compute the IRFFT (Default: first array dimension whose size does not equal 1)
    Returns
    -------
    x : Real-valued signal(s) 
        
    See Also
    --------
    `np.fft.ifft()`
    '''
    
    # verify y
    shape_y = y.shape
    # number of dimentions
    dim_y = len(shape_y)
    
    # limits to 2-dimention data
    assert dim_y<=2
        
    # check shape of y, and set n and dim defaults
    if dim_y == 1:
        dim_def = 0
    else:
        if shape_y[0] == 1:
            # shape [1, n_samples] (row vector)
            dim_def = 1
        elif shape_y[1] == 1:
            # shape [n_samples, 1] (column vector)
            dim_def = 0 
        else:
            # X is a 2D Matrix, a shape [n_samples, n_channels] is asummed
            dim_def = 0
     
    if dim is None:
        dim = dim_def
            
    # verify 'n' number-of-samples parameter
    if n is None:
        print('N not provided. Y is assumed be obtained from a signal X with even number fo samples')
        n_half = shape_y[dim]
        n = (n_half - 1) * 2
    
    # reconstruct missing half of Spectrum
    if np.mod(n,2) == 0:
        # number of samples is even
        n_half = (n / 2) + 1
        ix_limit = slice(1, -1 )
    else:
        # number of samples is odd
        n_half = (n + 1) / 2
        ix_limit = slice(1, None)

    if dim_y == 1:
        # spectrum in y is 1D
        y_neg = np.conj(np.flipud(y[ix_limit]))
        yc = np.concatenate((y, y_neg), axis=0)
    else:
        # check shape of y, and add negative frequencies
        if dim == 0:
            # spectra in y are column wise
            y_neg = np.conj(np.flipud(y[ix_limit, :]))
            yc = np.concatenate((y, y_neg), axis=0)
        else:
            # spectra in y are row-wise
            y_neg = np.conj(np.fliplr(y[:, ix_limit]))
            yc = np.concatenate((y, y_neg), axis=1)
        
    x = np.real(np.fft.ifft(yc, n, dim))
        
    return x


def rfft_psd(x, fs, n_fft=None, win_function = 'hamming', channel_names=None):
    """ This function computes the PSD for one or a set of REAL signals.
        
    Parameters
    ----------
    x  : 1D array with shape (n_samples) or
         2D array with shape (n_samples, n_channels)
    fs : Sampling frequency 
        in Hz
    n_fft : Number of samples to compute the FFT
            (Default = n_samples in array x)   
    win_function : Window function applied to the signal 
        (Default 'Hamming')
    channel_names : Names of the signals
        (Default Signal-XX with XX 1, 2, ... n_channels) 
    Returns
    -------
    psd_data : Dictionary with PSD data, with the elements:
       rFFT
           First half of the FFT(x) (u), scaled by the Window RMS       
       PSD
           Power Spectrum Density (u^2 / Hz) 
       fs
           Sampling frequency (Hz)
       freq_axis
           Frequency axis for rFFT and PSD (Hz)
       freq_delta
           Frequency axis step (Hz)
       n_samples
           Number of samples of the signal or signals 'x'
       n_fft
           Number of elements utilized to perform FFT
       win_function
           Window applied to the data in 'x'
       channel_names 
           Names of channels
    
    """

    # input 'x' as 2D matrix [samples, columns]
    try:
        x.shape[1]
    except IndexError:
        x = x[:, np.newaxis]
    
    # number of samples and number of channels
    n_samples, n_channels = x.shape
    
    # validate 'n_fft' argument
    if n_fft is None:
        n_fft = n_samples

    # generate default channel names, if needed
    if channel_names is None:
        channel_names = []
        for ic  in range (0 , n_channels):
            icp = ic + 1
            channel_names.append( str('Signal-%02d' % icp) )
            
    # windowing data
    win = scipy.signal.get_window(win_function, n_samples, fftbins=False)
    win.shape = (n_samples, 1)
    win_rms = np.sqrt(np.sum(np.square(win)) / n_samples)
    win_mat = np.tile(win, n_channels)
    x = np.multiply(x, win_mat)

    # real FFT with zero padding if n_fft ~= n_samples
    Xt = rfft(x, n_fft)
    # spectrum scaled by window RMS
    Xt = Xt / win_rms
    # power spectrum
    X_pwr = abs(np.multiply(Xt, np.conj(Xt)))
    X_pwr = X_pwr * (1/np.square(n_fft))

    # adjust for even and odd number of elements
    if n_fft % 2 != 0:
       # odd case
        n_freqs = (n_fft + 1) / 2
       # double all frequency components except DC component 
        X_pwr[1:, :] = X_pwr[1:, :] * 2
    
    else:
       # even case 
        n_freqs = (n_fft / 2) + 1
       # double all frequency components except DC and fs/2 components
        X_pwr[1:-1, :] = X_pwr[1:-1, :] * 2
    
    # frequency axis step
    f_delta = (fs / n_fft)
    # scale PSD with the frequency step
    psd = np.divide(X_pwr, f_delta)

    # frequency axis for spectrum
    n_freqs = int(n_freqs)
    f_axis = np.asarray(range(0, n_freqs)) * f_delta
    
    # output 'psd_data' dictionary
    psd_data = {}
    psd_data['rFFT'] = Xt
    psd_data['PSD'] = psd
    psd_data['fs'] = fs
    psd_data['freq_axis'] = f_axis
    psd_data['freq_delta'] = f_delta
    psd_data['n_samples'] = n_samples
    psd_data['n_fft'] = n_fft
    psd_data['win_function'] = win_function
    psd_data['channel_names'] = channel_names
    
    return psd_data

def irfft_psd(psd_data):
    """Compute the inverse PSD for one or a set of REAL signals.
         
     Parameters
     ----------
     psd_data : Structure with PSD data, created with rfft_psd()
 
     Returns
     -------
     x  : 1D array with shape (n_samples) or
          2D array with shape (n_samples, n_channels)
    """
    # Load data from PSD structure
    rFFT_data = psd_data['rFFT']
    f_ax = psd_data['freq_axis']
    fs = psd_data['fs']
    win_function = psd_data['win_function']
    n_samples = psd_data['n_samples']
    n_channels = rFFT_data.shape[1]
    
    # Find the number of elements used for the rFFT
    if f_ax[-1] < fs/2:
        # elements for FFT was odd
        n_fft = (len(f_ax) * 2) - 1
    elif f_ax[-1] - fs/2 < 1000 * np.finfo(np.float64).eps:
        # elements for FFT was even
        n_fft = (len(f_ax) - 1) * 2
    
    # Window RMS
    win = scipy.signal.get_window(win_function, n_samples, fftbins=False)
    win.shape = (n_samples, 1)
    win_rms = np.sqrt(np.sum(np.square(win)) / n_samples)
        
    # IRFFT
    X = rFFT_data * win_rms
    x_tmp = irfft(X, n_fft)
    
    # Keep only n_samples points
    x = x_tmp[0 : n_samples + 1, :]
    
    # Un-Windowing
    win_mat = np.tile(win, n_channels)
    x = np.divide(x, win_mat)
    
    return x

def strfft_spectrogram(x, fs, win_size, win_shift, n_fft=None, win_function='hamming', channel_names=None):
    """Compute the Short Time real FFT Spectrogram for one or a set of REAL signals 'x'.
        
    Parameters
    ----------
    x  : 1D array with shape (n_samples) or
         2D array with shape (n_samples, n_channels)
    fs : Sampling frequency 
         in Hz
    win_size :
        Size of the sliding window for STFFF (samples)
    win_shift :
        Shift between consecutive windows (samples)   
    n_fft : Number of samples to compute the FFT
            (Default = n_samples in array x)   
    win_function : Window function applied to the signal 
        (Default 'Hamming')
    channel_names : Names of the signals
        (Default Signal-XX with XX 1, 2, ... n_channels) 
    Returns
    -------
    spectrogram_data : Dictionary with Spectrogram data, with the elements:
       rFFT_spectrogram
           rFFT values for each window (u), scaled by the Window RMS       
       power_spectrogram :
           PSD values for each window (u^2 / Hz) 
       fs : 
           Sampling frequency (Hz)
       freq_axis :
           Frequency axis for rFFT and PSD (Hz)
       freq_delta :
           Frequency axis step (Hz)
       time_axis :
           Time axis for rFFT_spectrogram and power_spectrogram (s)       
       time_delta :
           Time axis step (s)
       win_size_samples :
           Size of the sliding window for STFFF (samples)
       win_shift_samples :
           Shift between consecutive windows (samples)   
       n_fft :
           Number of elements utilized to perform FFT    
       win_function :
           Window applied to the data in 'x'           
       n_windows :
           Number of ST windows
       n_samples :
           Number of samples of the signal or signals 'x'
       channel_names 
           Names of channels
    
    """
    
    # input 'x' as 2D matrix [samples, columns]
    try:
        x.shape[1]
    except IndexError:
        x = x[:, np.newaxis]
    
    # number of samples and number of channels
    n_samples, n_channels = x.shape
    
    # validate 'n_fft' argument    
    if n_fft is None:
        n_fft = win_size

    # round win_size and win_shift
    win_size = round(win_size)
    win_shift = round(win_shift)

    # time axis step for Spectrogram
    t_delta = win_shift / fs

    # Create time vector 'time_vct' for signal 'x'
    time_vct = np.array(range(0, np.size(x, 0))) / fs

    
    # epoch signal or signals 'x'
    x_epoched, _ , ix = epoching(x, win_size, win_size - win_shift)

    # time axis for Spectrogram
    t_ax = time_vct[ix]

    # spectrogram parameters
    n_samples_win, n_channels, n_windows = x_epoched.shape

    # generate default channel names, if needed
    if channel_names is None:
        channel_names = []
        for ic  in range (0 , n_channels):
            icp = ic + 1
            channel_names.append( str('Signal-%02d' % icp) )
                       
    # compute PSD per window
    for i_window in range(0, n_windows):
        # ith epoch of the signal or signals
        x_epoch = (x_epoched[:, :, i_window])
        psd_struct = rfft_psd(x_epoch, fs, n_fft, win_function, channel_names)
    
        # initialize arrays for spectrogram data
        if i_window == 0:
            # frequency Axis for spectrogram
            f_ax = psd_struct['freq_axis']
            # delta Frequency
            f_delta = psd_struct['freq_delta']
            # initialize 'rFFT_spectrogram' and 'pwr_spectrogram'
            rFFT_spectrogram = np.zeros((n_windows, len(f_ax), n_channels), dtype = complex)
            pwr_spectrogram  = np.zeros((n_windows, len(f_ax), n_channels))
        
        # rFFT data
        rFFT_spectrogram[i_window, :, :] = psd_struct['rFFT']
        # power data
        pwr_spectrogram[i_window, :, :] = psd_struct['PSD']

    # scale 'pwr_spectrogram' by number of windows and time delta
    pwr_spectrogram = pwr_spectrogram / (n_windows * t_delta)


    # output 'spectrogram_data' dictionary
    spectrogram_data = {}
    spectrogram_data['rFFT_spectrogram'] = rFFT_spectrogram
    spectrogram_data['power_spectrogram'] = pwr_spectrogram
    spectrogram_data['fs'] = fs
    spectrogram_data['freq_axis'] = f_ax
    spectrogram_data['freq_delta'] = f_delta
    spectrogram_data['time_axis'] = t_ax
    spectrogram_data['time_delta'] = t_delta
    spectrogram_data['win_size_samples'] = win_size
    spectrogram_data['win_shift_samples'] = win_shift
    spectrogram_data['n_fft'] = n_fft
    spectrogram_data['win_function'] = win_function
    spectrogram_data['n_windows'] = n_windows
    spectrogram_data['n_samples'] = n_samples    
    spectrogram_data['channel_names'] = channel_names

    return spectrogram_data

def istrfft_spectrogram(spectrogram_data):
    """Compute the inverse STFT spectrogram for one or a set of REAL signals.
        
    Parameters
    ----------
    spectrogram_data : Structure with STFT spectrogram data, created with strfft_spectrogram()
    Returns
    -------
    x  : 1D array with shape (n_samples) or
         2D array with shape (n_samples, n_channels)
    x_epoched   = Segments form the signal or set of signals utilized to
                  create the spectrogram in spectrogram_struct
    """
    # Load data from Spectrogram structure
    rFFT_data = spectrogram_data['rFFT_spectrogram']
    win_size = spectrogram_data['win_size_samples']
    win_shift = spectrogram_data['win_shift_samples']
    
    # Generate psd_struct, to use irfft_psd()
    psd_struct = {}
    psd_struct['fs'] = spectrogram_data['fs']
    psd_struct['channel_names'] = spectrogram_data['channel_names']
    psd_struct['freq_axis'] = spectrogram_data['freq_axis']
    psd_struct['win_function'] = spectrogram_data['win_function']
    psd_struct['n_samples'] = win_size
    
    # Initialize rFFT_slice and x_epoched variables
    (n_windows, n_freqs, n_channels) =  rFFT_data.shape
    rfft_slide = np.zeros((n_freqs, n_channels))
    x_epoched = np.zeros((win_size, n_channels, n_windows))
    
    for i_window in range(n_windows):
        # rFFT slice from spectrogram
        rfft_slide = rFFT_data[i_window, :, :]
        # Generate psd_struct, to use irfft_psd()
        psd_struct['rFFT'] = rfft_slide 
        # ifft_psd from the rFFT data recovers the signal or set of signals 'x'
        x_tmp = irfft_psd(psd_struct)
        x_epoched[:, :, i_window] = x_tmp

    # Merge epoched data
    x = iepoching(x_epoched, win_shift);
    
    return x, x_epoched

def wavelet_spectrogram(x, fs, n_cycles=6, freq_vct=None, channel_names=None):
    """Compute the Spectrogram using the Complex Morlet wavelet for one or a set of REAL signals 'x'. 
    
    Parameters
    ----------
    x  : 1D array with shape (n_samples) or
         2D array with shape (n_samples, n_channels)
    fs : Sampling frequency 
         in Hz
    n : Number of cicles inside the Gaussian curve 
        (Default 6)
    freq_vct : 1D array 
        with frequencies to compute the CWT (Default = [1 : 1 : fs/2] )
    channel_names : Names of the signals
        (Default Signal-XX with XX 1, 2, ... n_channels) 
    Returns
    -------
    spectrogram_data : Dictionary with Spectrogram data, with the elements:
       wavelet_coefficients
           Coefficients of the Wavelet transformation (u)       
       power_spectrogram :
           Power spectrogram (u^2 / Hz) 
       fs : 
           Sampling frequency (Hz)
       freq_axis :
           Frequency axis for rFFT and PSD (Hz)
       freq_delta :
           Frequency axis step (Hz)
       time_axis :
           Time axis for rFFT_spectrogram and power_spectrogram (s)       
       time_delta :
           Time axis step (s)
       n_cycles : 
           Number of cicles used inside the Gaussian curve 
       wavelet_kernels :
           Wavelet kernels used to obtain the wavelet coefficients
       n_samples :
           Number of samples of the signal or signals 'x'
       channel_names 
           Names of channels
    
    """
    # input 'x' as 2D matrix [samples, columns]
    try:
        x.shape[1]
    except IndexError:
        x = x[:, np.newaxis]
    
    # number of samples and number of channels
    n_samples, n_channels = x.shape
    
    # validate 'freq_vct' argument
    if freq_vct is None:
        freq_vct = np.array(range(1, int(np.floor(fs / 2) + 1)))
        
    # generate default channel names, if needed
    if channel_names is None:
        channel_names = []
        for ic  in range (0 , n_channels):
            icp = ic + 1
            channel_names.append( str('Signal-%02d' % icp) )
                
    # Time delta
    t_delta = 1 / fs
    
    # Frequency delta
    f_delta = freq_vct[1] - freq_vct[0]

    # Create time vector 'time_vct' for signal 'x'
    time_vct = np.array(range(0, np.size(x, 0))) / fs

    # Number of samples
    n_samples  = np.size(x, 0)

    # Wavelet transform
    wavelet_coefficients, wavelet_family = cmorlet_wavelet(x, fs, freq_vct, n_cycles)

    # Power from Wavelet coefficients
    power_spectrogram = np.square(np.abs(wavelet_coefficients))
    power_spectrogram = power_spectrogram * 2 / (fs * n_samples)

    # output 'spectrogram_data' dictionary
    spectrogram_data = {}
    spectrogram_data['wavelet_coefficients'] = wavelet_coefficients
    spectrogram_data['power_spectrogram'] = power_spectrogram
    spectrogram_data['fs'] = fs
    spectrogram_data['freq_axis'] = freq_vct
    spectrogram_data['freq_delta'] = f_delta
    spectrogram_data['time_axis'] = time_vct
    spectrogram_data['time_delta'] = t_delta
    spectrogram_data['n_cycles'] = n_cycles
    spectrogram_data['wavelet_kernels'] = wavelet_family    
    spectrogram_data['n_samples'] = n_samples
    spectrogram_data['channel_names'] = channel_names

    return spectrogram_data

def iwavelet_spectrogram(spectrogram_data):
    """ Compute the inverse CWT Spectrogram for one or a set of REAL signals.
        
    Parameters
    ----------
    spectrogram_data : Structure with CWT Spectrogram data, created with wavelet_spectrogram()
    Returns
    -------
    x  : 1D array with shape (n_samples) or
         2D array with shape (n_samples, n_channels)
    x_epoched   = Segments form the signal or set of signals utilized to
                  create the spectrogram in spectrogram_struct
    """
    
    # compute the scaling factor for each wavelet kernel
    s = spectrogram_data['n_cycles'] / ( 2 * np.pi * spectrogram_data['freq_axis'])
    A = 1. / ((s**2) * np.pi) ** (1./4)


    x_tmp = np.real(spectrogram_data['wavelet_coefficients'])

    # compute the mean across scaled "filtered" signals
    for ix, a in enumerate(A):
        x_tmp[:, ix, :] = x_tmp[:, ix, :] / a 
    
    x = np.mean(x_tmp, axis = 1) 
    
    #x = squeeze(mean( bsxfun(@rdivide, real(spectrogram_data.wavelet_coefficients) , A ), 2));

    return x

def strfft_modulation_spectrogram(x, fs, win_size, win_shift, fft_factor_y=None, win_function_y='hamming', fft_factor_x=None, win_function_x='hamming', channel_names=None):
    """Compute the Modulation Spectrogram using the Complex Morlet wavelet for one or a set of REAL signals 'x'.
        
    Parameters
    ----------
    x  : 1D array with shape (n_samples) or
         2D array with shape (n_samples, n_channels)
    fs : Sampling frequency 
         in Hz
    win_size :
        Size of the sliding window for STFFF (samples)
    win_shift :
        Shift between consecutive windows (samples)   
    fft_factor_y : Number of elements to perform the 1st FFT is given as:
        n_fft_y  = fft_factor_y * n_samples, (default, fft_factor_y = 1)
    win_function_y : Window to apply in the 1st FFT 
        (Default 'Hamming')
    fft_factor_x : Number of elements to perform the 2nd FFT is given as:
        n_fft_x  = fft_factor_x * n_samples, (default, fft_factor_x = 1)
    win_function_x : Window to apply in the 2nd rFFT 
        (Default 'Hamming')   
    n_fft : Number of samples to compute the FFT
        (Default = n_samples in array x)   
    channel_names : Names of the signals
        (Default Signal-XX with XX 1, 2, ... n_channels) 
    Returns
    -------
    modulation_spectrogram_data : Dictionary with Modulation Spectrogram data, with the elements:
       rFFT_modulation_spectrogram
           rFFT values for each window (u), scaled by the Window RMS       
       power_modulation_spectrogram :
           Power modulation spectrogram (u^2 / Hz) 
       fs : 
           Sampling frequency (Hz)
       fs_mod : 
           Sampling frequency of modulation-frequency (Hz)         
       freq_axis :
           Frequency axis for rFFT and PSD (Hz)
       freq_delta :
           Frequency axis step (Hz)
       freq_mod_axis :
           Modulation-frequency axis for rFFT_modspec and pwr_modspec (Hz)     
       freq_mod_delta :
           Modulation-frequency step (Hz)
       win_size_samples :
           Size of the sliding window for STFFF (samples)
       win_shift_samples :
           Shift between consecutive windows (samples)   
       n_fft_y :
           Number of elements utilized to perform the 1st FFT
       n_fft_x :
           Number of elements utilized to perform the 2nd FFT
       win_function_y :
           Window to apply in the 1st rFFT            
       win_function_x :
           Window to apply in the 2nd rFFT                      
       n_windows :
           Number of ST windows
       n_samples :
           Number of samples of the signal or signals 'x'
       spectrogram_data : 
           Dictionary with Spectrogram data
       channel_names :
           Names of channels
    
    """
    # input 'x' as 2D matrix [samples, columns]
    try:
        x.shape[1]
    except IndexError:
        x = x[:, np.newaxis]
    
    # number of samples and number of channels
    n_samples, n_channels = x.shape
    
    # validate 'fft_factor_y' argument    
    if fft_factor_y is None:
        fft_factor_y = 1
        
    # validate 'fft_factor_x' argument    
    if fft_factor_x is None:
        fft_factor_x = 1
        
    # number of elements for the 1st FFT
    n_fft_y = fft_factor_y * win_size

    
    # compute STFFT spectrogram
    spectrogram_data = strfft_spectrogram(x, fs, win_size, win_shift, n_fft_y, win_function_y, channel_names)
    n_windows, n_freqs, n_channels = spectrogram_data['rFFT_spectrogram'].shape
    # Number of elements for the 2nd FFT
    n_fft_x =  fft_factor_x * n_windows

    # generate default channel names, if needed
    if channel_names is None:
        channel_names = []
        for ic  in range (0 , n_channels):
            icp = ic + 1
            channel_names.append( str('Signal-%02d' % icp) )
            
    # modulation sampling frequency
    fs_mod = 1 / (win_shift / fs)

    # the AM analysis is made in the Amplitude derived from the Power Spectrogram
    for i_channel in range(0, n_channels):
        # data to generate the Modulation Spectrogram
        spectrogram_1ch = np.sqrt(spectrogram_data['power_spectrogram'][:,:,i_channel]) 

        # compute 'rfft_psd' on each frequency timeseries
        mod_psd_struct = rfft_psd(spectrogram_1ch, fs_mod, n_fft_x, win_function_x, channel_names )
    
        if i_channel == 0:
            # modulation frequency axis
            fmod_ax = mod_psd_struct['freq_axis']
            # modulation frequency delta
            fmod_delta = mod_psd_struct['freq_delta']
    
            # initialize 'rFFT_modspec'  and 'pwr_modspec'
            n_freqsmod = len(fmod_ax)
            rFFT_modspec = np.zeros((n_freqs, n_freqsmod ,n_channels), dtype = complex)
            pwr_modspec  = np.zeros((n_freqs, n_freqsmod ,n_channels))

        # rFFT data
        rFFT_modspec[:, :, i_channel] = mod_psd_struct['rFFT'].transpose()
        # power data
        pwr_modspec[:, :, i_channel] = mod_psd_struct['PSD'].transpose()

    # scale 'pwr_modspec' by modulation-frequency delta
    pwr_modspec = pwr_modspec / fmod_delta

    # output 'modulation_spectrogram_data' structure
    modulation_spectrogram_data = {}
    modulation_spectrogram_data['rFFT_modulation_spectrogram'] = rFFT_modspec
    modulation_spectrogram_data['power_modulation_spectrogram'] = pwr_modspec
    modulation_spectrogram_data['fs'] = fs
    modulation_spectrogram_data['fs_mod'] = fs_mod
    modulation_spectrogram_data['freq_axis'] = spectrogram_data['freq_axis']
    modulation_spectrogram_data['freq_delta'] = spectrogram_data['freq_delta']
    modulation_spectrogram_data['freq_mod_axis'] = fmod_ax
    modulation_spectrogram_data['freq_mod_delta'] = fmod_delta
    modulation_spectrogram_data['win_size_samples'] = win_size
    modulation_spectrogram_data['win_shift_samples'] = win_shift
    modulation_spectrogram_data['n_fft_y'] = n_fft_y
    modulation_spectrogram_data['n_fft_x'] = n_fft_x
    modulation_spectrogram_data['win_function_y'] = win_function_y
    modulation_spectrogram_data['win_function_x'] = win_function_x
    modulation_spectrogram_data['n_windows'] = n_windows 
    modulation_spectrogram_data['n_samples'] = spectrogram_data['n_samples'] 
    modulation_spectrogram_data['spectrogram_data'] = spectrogram_data
    modulation_spectrogram_data['channel_names'] = channel_names
    
    return modulation_spectrogram_data

def istrfft_modulation_spectrogram(modulation_spectrogram_data):
    """ Compute the inverse STFT-based modulation spectrogram for one or a set of REAL signals.
         
     Parameters
     ----------
     modulation_spectrogram_data : Structure with STFT-based modulation spectrogram data, 
           created with strfft_modulation_spectrogram()
 
     Returns
     -------
     x  : 1D array with shape (n_samples) or
          2D array with shape (n_samples, n_channels)
    
    """
    # Number of channels from Modspectrogram structure
    n_channels = modulation_spectrogram_data['rFFT_modulation_spectrogram'].shape[2]
    
    # Prepare psd_tmp_data to perform irFFT on Modulation Spectogram
    psd_tmp_data = {}
    psd_tmp_data['freq_axis'] = modulation_spectrogram_data['freq_mod_axis']
    psd_tmp_data['fs'] = modulation_spectrogram_data['fs_mod']
    psd_tmp_data['win_function'] = modulation_spectrogram_data['win_function_x']
    psd_tmp_data['n_samples'] = modulation_spectrogram_data['n_windows']


    for i_channel in range(n_channels):
        # Slide with the rFFT coeffients of the 2nd FFT 
        psd_tmp_data['rFFT'] = np.transpose(modulation_spectrogram_data['rFFT_modulation_spectrogram'][:,:,i_channel])   
        # Recovers the Square Root of the Power Spectrogram
        sqrt_pwr_spectrogram = irfft_psd(psd_tmp_data)
        # Power Spectrogram
        pwr_spectrogram = sqrt_pwr_spectrogram ** 2
        # Scale Power Spectrogram by (n_windows * time_delta)
        pwr_spectrogram = pwr_spectrogram * modulation_spectrogram_data['spectrogram_data']['n_windows'] * modulation_spectrogram_data['spectrogram_data']['time_delta']
        # Scale Power Spectrogram by (freq_delta)
        pwr_spectrogram = pwr_spectrogram * modulation_spectrogram_data['spectrogram_data']['freq_delta']
        # Scale Power Spectrogram by the number of samples used
        pwr_spectrogram = pwr_spectrogram / (1 / modulation_spectrogram_data['spectrogram_data']['n_fft'] ** 2)
        # Divde by 2 all the elements except DC and the Nyquist point (in even case)  
        pwr_spectrogram = pwr_spectrogram / 2
        pwr_spectrogram[:, 0] = pwr_spectrogram[:, 0] * 2
        if np.mod(modulation_spectrogram_data['spectrogram_data']['n_fft'], 2) == 0:
            # NFFT was even, then 
            pwr_spectrogram[:, -1] = pwr_spectrogram[:, -1] * 2
        spectrogram_abs = np.sqrt(pwr_spectrogram)
        # Recovers the Angle values of the Spectrogram
        spectrogram_angle = np.angle(modulation_spectrogram_data['spectrogram_data']['rFFT_spectrogram'][:,:,i_channel])
        # Creates the rFFT coefficients of the 1st FFTs
        modulation_spectrogram_data['spectrogram_data']['rFFT_spectrogram'][:,:,i_channel] = spectrogram_abs * np.exp(1j * spectrogram_angle )   

    # Recovers the origial signal or set of signals
    x = istrfft_spectrogram(modulation_spectrogram_data['spectrogram_data'])[0]
    
    return x

def wavelet_modulation_spectrogram(x, fs, n_cycles=6, freq_vct=None, fft_factor_x=1, win_function_x='hamming', channel_names=None):
    """Compute the Modulation Spectrogram using the Wavelet for one or a set of REAL signals 'x'.
        
    Parameters
    ----------
    x  : 1D array with shape (n_samples) or
         2D array with shape (n_samples, n_channels)
    fs : Sampling frequency 
         in Hz
    n : Number of cicles inside the Gaussian curve 
        (Default 6)
    freq_vct : 1D array 
        with frequencies to compute the CWT (Default = [1 : 1 : fs/2] )
    fft_factor_x : Number of elements to perform the FFT is given as:
        n_fft_x  = fft_factor_x * n_samples, (default, fft_factor_x = 1)
    win_function_x : Window to apply in the rFFT 
        (Default 'Hamming')   
    channel_names : Names of the signals
        (Default Signal-XX with XX 1, 2, ... n_channels) 
    Returns
    -------
    modulation_spectrogram_data : Dictionary with Modulation Spectrogram data, with the elements:
       rFFT_modulation_spectrogram
           rFFT values for each window (u), scaled by the Window RMS       
       power_modulation_spectrogram :
           Power modulation spectrogram (u^2 / Hz) 
       fs : 
           Sampling frequency (Hz)
       fs_mod : 
           Sampling frequency of modulation-frequency (Hz)         
       freq_axis :
           Frequency axis for rFFT and PSD (Hz)
       freq_delta :
           Frequency axis step (Hz)
       freq_mod_axis :
           Modulation-frequency axis for rFFT_modspec and pwr_modspec (Hz)     
       freq_mod_delta :
           Modulation-frequency step (Hz)
       n_fft_x :
           Number of elements utilized to perform the FFT
       win_function_x :
           Window to apply in the 2nd rFFT                      
       n_samples :
           Number of samples of the signal or signals 'x'
       spectrogram_data : 
           Dictionary with Spectrogram data
       channel_names :
           Names of channels
    
    """
    # input 'x' as 2D matrix [samples, columns]
    try:
        x.shape[1]
    except IndexError:
        x = x[:, np.newaxis]
    
    # number of samples and number of channels
    n_samples, n_channels = x.shape    
    
    # generate default channel names, if needed
    if channel_names is None:
        channel_names = []
        for ic  in range (0 , n_channels):
            icp = ic + 1
            channel_names.append( str('Signal-%02d' % icp) )
    
    spectrogram_data = wavelet_spectrogram(x, fs, n_cycles, freq_vct, channel_names)
    n_windows, n_freqs, n_channels =  spectrogram_data['wavelet_coefficients'].shape

    # number of elements for FFT of the spectrogram
    n_fft_x =  fft_factor_x * n_windows    

    fs_mod = fs

    # the AM analysis is made in the Amplitude derived from the Power Spectrogram
    for i_channel in range(0, n_channels):
        # data to generate the Modulation Spectrogram
        spectrogram_1ch = np.sqrt(spectrogram_data['power_spectrogram'][:, :, i_channel])
        # Compute rfft_psd on each frequency timeseries
        psd_dict = rfft_psd(spectrogram_1ch, fs, n_fft_x)
    
        rfft_result = psd_dict['rFFT']
        rfft_psd_res = psd_dict['PSD']
       
        if i_channel == 0:
            # modulation frequency axis
            fmod_ax = psd_dict['freq_axis']
            # modulation frequency delta
            fmod_delta = psd_dict['freq_delta']
            n_freqsmod = np.size(fmod_ax)
            # initialize 'rFFT_modspec'  and 'pwr_modspec'
            rfft_modspec = np.zeros((n_freqs, n_freqsmod, n_channels), dtype = complex)
            pwr_modspec  = np.zeros((n_freqs, n_freqsmod, n_channels))
    
        # rFFT data
        rfft_modspec[:, :, i_channel] = np.transpose(rfft_result)            
        # power data
        pwr_modspec[:, :, i_channel] = np.transpose(rfft_psd_res)
    
    # scale 'pwr_modspec' by modulation-frequency delta    
    pwr_modspec = pwr_modspec / fmod_delta

    # output 'modulation_spectrogram_data' dictionary
    modulation_spectrogram_data = {}
    modulation_spectrogram_data['rFFT_modulation_spectrogram'] = rfft_modspec
    modulation_spectrogram_data['power_modulation_spectrogram'] = pwr_modspec
    modulation_spectrogram_data['fs'] = fs
    modulation_spectrogram_data['fs_mod'] = fs_mod
    modulation_spectrogram_data['freq_axis'] = spectrogram_data['freq_axis']
    modulation_spectrogram_data['freq_delta'] = spectrogram_data['freq_delta']
    modulation_spectrogram_data['freq_mod_axis'] = fmod_ax
    modulation_spectrogram_data['freq_mod_delta'] = fmod_delta
    modulation_spectrogram_data['n_fft_x'] = n_fft_x
    modulation_spectrogram_data['win_function_x'] = win_function_x
    modulation_spectrogram_data['n_samples'] = spectrogram_data['n_samples'] 
    modulation_spectrogram_data['spectrogram_data'] = spectrogram_data
    modulation_spectrogram_data['channel_names'] = channel_names
    
    return modulation_spectrogram_data

def iwavelet_modulation_spectrogram(modulation_spectrogram_data):
    """ Compute the inverse CWT-based modulation spectrogram for one or a set of REAL signals.
        
    Parameters
    ----------
    modulation_spectrogram_data : Structure with CWT-based modulation spectrogram data, 
          created with wavelet_modulation_spectrogram()
    Returns
    -------
    x  : 1D array with shape (n_samples) or
         2D array with shape (n_samples, n_channels)
    """
    # Number of channels from Modspectrogram structure
    n_channels = modulation_spectrogram_data['rFFT_modulation_spectrogram'].shape[2]
    
    # Prepare psd_tmp_data to perform irFFT on Modulation Spectogram
    psd_tmp_data = {}
    psd_tmp_data['freq_axis'] = modulation_spectrogram_data['freq_mod_axis']
    psd_tmp_data['fs'] = modulation_spectrogram_data['fs_mod']
    psd_tmp_data['win_function'] = modulation_spectrogram_data['win_function_x']
    psd_tmp_data['n_samples'] = modulation_spectrogram_data['n_samples']
    
        
    for i_channel in range(n_channels):
        # Slide with the rFFT coeffients of the 2nd FFT 
        psd_tmp_data['rFFT'] = np.transpose(modulation_spectrogram_data['rFFT_modulation_spectrogram'][:,:,i_channel])   
        # Recovers the Square Root of the Power Spectrogram
        sqrt_pwr_spectrogram = irfft_psd(psd_tmp_data)
        
        # Recovers the Magnitude of the Wavelet Coefficents
        pwr_spectrogram = sqrt_pwr_spectrogram ** 2
        pwr_spectrogram = pwr_spectrogram * modulation_spectrogram_data['fs_mod'] *  modulation_spectrogram_data['n_samples']
        pwr_spectrogram = pwr_spectrogram / 2
        spectrogram_abs = np.sqrt(pwr_spectrogram)
            
        # Recovers the Angle values of the Spectrogram
        spectrogram_angle = np.angle(modulation_spectrogram_data['spectrogram_data']['wavelet_coefficients'][:,:,i_channel])
        
        # Creates the rFFT coefficients of the 1st FFTs
        modulation_spectrogram_data['spectrogram_data']['wavelet_coefficients'][:,:,i_channel] = spectrogram_abs * np.exp(1j * spectrogram_angle )   
    
    # Recovers the origial signal or set of signals
    x = iwavelet_spectrogram(modulation_spectrogram_data['spectrogram_data'])

    return x

def plot_spectrogram_data(spectrogram_data, ix=None, t_range=None, f_range=None, c_range=None, c_map='viridis'):
    """ Plot the Power Spectrogram related to the `spectrogram_data`
        
    Parameters
    ----------
    spectrogram_data : 
        Dictionary with Spectrogram data
    ix : Index of the signal (channel) to plot
        (Default, all the channels, a new figure for each)
    t_range : Time range
        (Default [minimum time, maximum time])
    f_range : Frequency range
        (Default [minimum frequency, maximum frequency])
    c_range : Color (power) range
        (Default [mean power, maximum power])
    c_map : Colot Map
        (Default viridis)
   
    Returns
    -------
    If only a plot is requested, it is plotted in the existen axes (created if needed)
    If many plots are requested, a new figure is created for each plot
    
    """    
    
    def plot_one_spectrogram(ax, X_pwr, t_ax, f_ax, title_str, t_range, f_range, c_range, c_map):
        """
        Plots ONLY ONE Spectrogram
        """
        T, F = np.meshgrid(t_ax, f_ax) 
        X_plot = 10 * np.log10(X_pwr[:,:].transpose() + np.finfo(float).eps) 
        pmesh = plt.pcolormesh(T,F,X_plot, cmap=c_map)
    
        # Major and Minor ticks
        ax = plt.gca()
        ax.xaxis.set_major_locator(ticker.AutoLocator())
        ax.xaxis.set_minor_locator(ticker.AutoMinorLocator())
        ax.yaxis.set_major_locator(ticker.AutoLocator())
        ax.yaxis.set_minor_locator(ticker.AutoMinorLocator())
             
        plt.xlabel('fime (s)')
        plt.ylabel('frequency (Hz)')
    
    
        if t_range is not None:
            xlim = t_range
        else:
            xlim = t_ax
    
        if f_range is not None:
            ylim = f_range
        else:
            ylim = f_ax
    
        # set the limits of the plot to the limits of the data
        plt.axis([xlim.min(), xlim.max(), ylim.min(), ylim.max()])
          
        if c_range is not None:
            clim = c_range
        else:
            clim = np.array([np.mean(X_plot), np.amax(X_plot)])  
    
        pmesh.set_clim(vmin=clim[0], vmax=clim[1])
    
        plt.colorbar()
        plt.title(title_str)
        plt.draw()

    
    # validate 'ix' argument    
    if ix is None:
        ix  = range(0, spectrogram_data['power_spectrogram'].shape[2])
    elif np.isscalar(ix):
        ix = np.array([ix])
        
    # Check if ix has ONLY one element
    if len(ix) == 1:
        new_figure = False
        # Retrieve Current Axes handle from the Current Figure, if there is not
        # Current Figure, it's generated here       
        ax = plt.gca()
    else:
        new_figure = True

    for i_channel in ix:      
        if new_figure:
            plt.figure()
            ax = plt.gca()
        plot_one_spectrogram(ax, 
                             spectrogram_data['power_spectrogram'][:, :, i_channel], 
                             spectrogram_data['time_axis'], 
                             spectrogram_data['freq_axis'], 
                             spectrogram_data['channel_names'][i_channel],
                             t_range, f_range, c_range, c_map)    
    


def plot_modulation_spectrogram_data(modulation_spectrogram_data, ix=None, f_range=None, modf_range=None, c_range=None, c_map='viridis'):
    """ Plot the Power Modulation Spectrogram related to the `modulation_spectrogram_data`
        
    Parameters
    ----------
    modulation_spectrogram_data : 
        Dictionary with Modulation Spectrogram data
    ix : Index of the signal (channel) to plot
        (Default, all the channels, a new figure for each)
    f_range : Frequency range
        (Default [minimum frequency, maximum frequency])
    fm_range : Modulation frequency range
        (Default [minimum mod_frequency, maximum mod_frequency])
    c_range : Color (power) range
        (Default [mean power, maximum power])
    c_map : Colot Map
        (Default viridis)
   
    Returns
    -------
    If only a plot is requested, it is plotted in the existen axes (created if needed)
    If many plots are requested, a new figure is created for each plot
    
    """     
    
    def plot_one_modulation_spectrogram(ax, X_pwr, f_ax, modf_ax, title_str, f_range, modf_range, c_range, c_map):
        """
        Plots ONLY ONE Modulation Spectrogram
        """
        MF, F = np.meshgrid(modf_ax, f_ax) 
        X_plot = 10 * np.log10(X_pwr[:,:] + np.finfo(float).eps) 
        pmesh = plt.pcolormesh(MF, F, X_plot, cmap =c_map)
    
        # Major and Minor ticks
        ax = plt.gca()
        ax.xaxis.set_major_locator(ticker.AutoLocator())
        ax.xaxis.set_minor_locator(ticker.AutoMinorLocator())
        ax.yaxis.set_major_locator(ticker.AutoLocator())
        ax.yaxis.set_minor_locator(ticker.AutoMinorLocator())
             
        plt.xlabel('modulation frequency (Hz)')
        plt.ylabel('conventional frequency (Hz)')
    
    
        if modf_range is not None:
            xlim = modf_range
        else:
            xlim = modf_ax
    
        if f_range is not None:
            ylim = f_range
        else:
            ylim = f_ax
    
        # set the limits of the plot to the limits of the data
        plt.axis([xlim.min(), xlim.max(), ylim.min(), ylim.max()])
          
        if c_range is not None:
            clim = c_range
        else:
            clim = np.array([np.mean(X_plot), np.amax(X_plot)])  
    
        pmesh.set_clim(vmin=clim[0], vmax=clim[1])
    
        plt.colorbar()
        plt.title(title_str)
        plt.draw()
      
    # validate 'ix' argument    
    if ix is None:
        ix  = range(0, modulation_spectrogram_data['power_modulation_spectrogram'].shape[2])
    elif np.isscalar(ix):
        ix = np.array([ix])
    
    # Check if ix has ONLY one element
    if len(ix) == 1:
        new_figure = False
        # Retrieve Current Axes handle from the Current Figure, if there is not
        # Current Figure, it's generated here       
        ax = plt.gca()
    else:
        new_figure = True

    for i_channel in ix:      
        if new_figure:
            plt.figure()
            ax = plt.gca()
        plot_one_modulation_spectrogram(ax, 
                             modulation_spectrogram_data['power_modulation_spectrogram'][:, :, i_channel], 
                             modulation_spectrogram_data['freq_axis'], 
                             modulation_spectrogram_data['freq_mod_axis'], 
                             modulation_spectrogram_data['channel_names'][i_channel],
                             f_range, modf_range, c_range, c_map)    


def plot_psd_data(psd_data, ix=None, p_range=None, f_range=None):
    """ Plot the PSD related to the `psd_data`
        
    Parameters
    ----------
    psd_data : 
        Dictionary with PSD data
    ix : Index of the signal (channel) to plot
        (Default, all the channels, a new figure for each)
    p_range : Power range
        (Default [minimum power, maximum power])
    f_range : Frequency range
        (Default [minimum frequency, maximum frequency])
    
    Returns
    -------
    If only a plot is requested, it is plotted in the existen axes (created if needed)
    If many plots are requested, a new figure is created for each plot
    
    """   
       
    def plot_one_psd(ax, X_pwr, f_ax, title_str, p_range, f_range):
        """
        Plots ONLY ONE PSD
        """    
        X_plot = 10 * np.log10(X_pwr + np.finfo(float).eps) 
        plt.plot(f_ax, X_plot)
    
        # Major and Minor ticks
        ax = plt.gca()
        ax.xaxis.set_major_locator(ticker.AutoLocator())
        ax.xaxis.set_minor_locator(ticker.AutoMinorLocator())
        ax.yaxis.set_major_locator(ticker.AutoLocator())
        ax.yaxis.set_minor_locator(ticker.AutoMinorLocator())
             
        plt.xlabel('frequency (Hz)')
        plt.ylabel('power (dB/Hz)')
    
    
        if f_range is not None:
            xlim = f_range
        else:
            xlim = f_ax
    
        if p_range is not None:
            ylim = p_range
        else:
            ylim = X_plot
    
        # set the limits of the plot to the limits of the data
        plt.axis([xlim.min(), xlim.max(), ylim.min(), ylim.max()])
    
        plt.title(title_str)
        plt.draw()
       
    # validate 'ix' argument    
    if ix is None:
        ix  = range(0, psd_data['PSD'].shape[1])
    elif np.isscalar(ix):
        ix = np.array([ix])
    
    # Check if ix has ONLY one element
    if len(ix) == 1:
        new_figure = False
        # Retrieve Current Axes handle from the Current Figure, if there is not
        # Current Figure, it's generated here       
        ax = plt.gca()
    else:
        new_figure = True

    for i_channel in ix:      
        if new_figure:
            plt.figure()
            ax = plt.gca()
        plot_one_psd(ax, 
                     psd_data['PSD'][:, i_channel], 
                     psd_data['freq_axis'], 
                     psd_data['channel_names'][i_channel],
                     p_range, f_range)    


def plot_signal(x, fs, name=None):
    """Behaves as matplotlib.pyplot.plot(x) but X axis is definded by `fs` [Hz]
    
    Parameters
    ----------
    x : 
        1D or 2D Signals as column vectors 
    fs :
        Sampling frequency in Hz
    name :
        Name of the signal (Default 'Signal-01')
    """
    
    # Create time vector
    time_vector = np.arange(x.shape[0])/fs
    
    plt.plot(time_vector,x)
    plt.xlabel('time (s)')
    plt.xlim([time_vector.min(), time_vector.max()])
    
    if name is None:
        name = 'Signal-01'
    
    plt.title(name)
    plt.draw()

def msqi_ama(x, fs):
    """
    Computes the Modulation Spectrum-Based ECG Quality Index (MSQI) for one or 
    many ECG signals defined in x, sampled with a sampling frequency fs
    Parameters
    ----------
    x  : 1D array with shape (n_samples) or
         2D array with shape (n_samples, n_signals)
    fs : Sampling frequency in Hz
    Returns
    -------
    msqi_value : MSQI value or values 
    hr_value   : HR values or values
    modulation_spectrogram : Structure or structures of modulation spectrogram
    
    See
    --------
    MS-QI: A Modulation Spectrum-Based ECG Quality Index for Telehealth Applications
    http://ieeexplore.ieee.org/document/6892964/
    
    D. P. Tobon V., T. H. Falk, and M. Maier, "MS-QI:  A  Modulation
    Spectrum-Based ECG Quality Index for Telehealth Applications", IEEE
    Transactions on Biomedical Engineering, vol. 63, no. 8, pp. 1613-1622,
    Aug. 2016    
    """
    
    # test ecg shape
    try:
        x.shape[1]
    except IndexError:
        x = x[:, np.newaxis]
    
    # Empirical values for the STFFT transformation
    win_size_sec  = 0.125   #seconds
    win_over_sec  = 0.09375 #seconds
    nfft_factor_1 = 16
    nfft_factor_2 = 4

    win_size_smp = int(win_size_sec * fs) #samples
    win_over_smp = int(win_over_sec * fs) #samples
    win_shft_smp = win_size_smp - win_over_smp

    # Computes Modulation Spectrogram
    modulation_spectrogram = strfft_modulation_spectrogram(x, fs, win_size_smp, 
            win_shft_smp, nfft_factor_1, 'cosine', nfft_factor_2, 'cosine' )
    
    # Find fundamental frequency (HR)
    # f = (0, 40)Hz
    ix_f_00 = (np.abs(modulation_spectrogram['freq_axis'] -  0)).argmin(0)  
    ix_f_40 = (np.abs(modulation_spectrogram['freq_axis'] - 40)).argmin(0) + 1
    
    # Look for the maximum only from 0.6 to 3 Hz (36 to 180 bpm)
    valid_f_ix = np.logical_or(modulation_spectrogram['freq_mod_axis'] < 0.66 , modulation_spectrogram['freq_mod_axis'] > 3)
    
    # number of epochs
    n_epochs = modulation_spectrogram['power_modulation_spectrogram'].shape[2]
    
    msqi_vals = np.zeros(n_epochs)
    hr_vals   = np.zeros(n_epochs)
    
    for ix_epoch in range(n_epochs):
        B = np.sqrt(modulation_spectrogram['power_modulation_spectrogram'][:, :, ix_epoch])
    
        # Scale to maximun of B
        B = B / np.max(B)
    
        # Add B in the conventional frequency axis from 0 to 40 Hz
        tmp = np.sum(B[ix_f_00:ix_f_40, :], axis=0)
    
        # Look for the maximum only from 0.6 to 3 Hz (36 to 180 bpm)
        tmp[valid_f_ix] = 0
        ix_max = np.argmax(tmp)     
        freq_funda = modulation_spectrogram['freq_mod_axis'][ix_max]        
                   
        # TME
        tme = np.sum(B)
    
        eme = 0
        for ix_harm in range(1, 5):
            ix_fm = (np.abs(modulation_spectrogram['freq_mod_axis'] - (ix_harm * freq_funda) )).argmin(0) 
            ix_b = int(round(.3125 / modulation_spectrogram['freq_mod_delta'] ))  # 0.3125Hz, half lobe
            # EME
            eme = eme + np.sum(B[ 0 : ix_f_40, ix_fm - ix_b : ix_fm + ix_b + 1 ])  
        
        # RME
        rme = tme - eme
        # MS-QI
        msqi_vals[ix_epoch] = eme / rme
        # HR
        hr_vals[ix_epoch] = freq_funda * 60
         
    return (msqi_vals, hr_vals, modulation_spectrogram)

In [ ]:
def overall_plot(signal,name,spectro_w,t_inter,length = data.signal_length,fs = data.sampling_frequency):
    x = np.array(range(0,int(data.signal_length)))
    x = x/data.sampling_frequency 
    look_up = np.array(t_inter)
    fig,ax = plt.subplots(nrows = 1,ncols = 2, figsize = (20,10))
    ax[0].plot(x,signal)
    ax[0].set_title(f"Full signal of Lead {name.decode('utf8')}")
    ax[0].grid()
    ax[1].plot(x,signal)
    ax[1].set_title(f"Close up signal of Lead {name.decode('utf8')}")
    ax[1].grid()
    ax[1].set_xlim(look_up)
    plt.figure()
    plt.title(f"Wavelet Scalogram for the full signal of lead {name.decode('utf8')}")
    plot_spectrogram_data(spectro_w,f_range=np.array([0,120]))
    plt.show()
    plt.figure()
    plt.title(f"Wavelet Scalogram for the full signal of lead {name.decode('utf8')}")
    plot_spectrogram_data(spectro_w,f_range=np.array([0,120]),t_range=look_up)
    plt.show()

for i in ECG_lead:
    MSSRw = wavelet_modulation_spectrogram(dico_ECG[i],sample.sampling_frequency)
    spectrogram_w = wavelet_spectrogram(dico_ECG[i],sample.sampling_frequency)
    overall_plot(dico_ECG[i],i,spectrogram_w,[1,3])
    plt.figure()
    print(f"Wavelet MSSR for lead {i.decode('utf8')}")
    plot_modulation_spectrogram_data(MSSRw,modf_range=np.array([0,10]),f_range=np.array([0,120]))
    plt.show()
    ##MS-QI
    MSQI,_,_ = msqi_ama(dico_ECG[i],sample.sampling_frequency)
    print(f"MS QI of the Lead {i.decode('utf8')} : ",MSQI)



In [ ]:
ecg_syn = scipy.misc.electrocardiogram()
fs = 360
time = np.arange(ecg_syn.size) / fs
#interval = np.array([[9,10.2],[20,25],[46.5,50],[60,65],[90,95],[110,115],[130,135],[180,185],[207,215],[100,106]])
interval = np.array([[60,67],[90,97],[110,117],[130,138],[180,187],[207,217],[100,108]])
for inter in interval:
    MSSRw = wavelet_modulation_spectrogram(ecg_syn[np.logical_and(time>inter[0], time<inter[1])],fs)
    spectrogram_w = wavelet_spectrogram(ecg_syn,fs)
    plot_signal(ecg_syn,fs,name = f"ECG synth between [{inter[0]};{inter[1]}] ")
    plt.xlim([inter[0],inter[1]])
    plt.grid()
    plt.show()
    plt.figure()
    print(f"Wavelet Scalogram for ECG syn between [{inter[0]};{inter[1]}]")
    plot_spectrogram_data(spectrogram_w,f_range=np.array([0,120]),t_range=np.array([inter[0],inter[1]]))
    plt.show()
    print(f"Wavelet MSSR for ECG syn between [{inter[0]};{inter[1]}]")
    plot_modulation_spectrogram_data(MSSRw,modf_range=np.array([0,10]),f_range=np.array([0,120]))
    plt.show()


In [ ]:
##DMD representation
from pydmd import DMD,HODMD

i = ECG_lead[0]
signal  = dico_ECG[i]
signal_cut = signal[0:500]

test_hodmd = HODMD(svd_rank=0, exact=True, opt=True, d=30).fit(signal_cut)
#test_hodmd = DMD(svd_rank=15).fit(X)
plt.figure()
plot_ECG_signal(signal,i)
plt.show()
for eig in test_hodmd.eigs:
    print('Eigenvalue {}: distance from unit circle {}'.format(eig, np.abs(eig.imag**2+eig.real**2 - 1)))

test_hodmd.plot_eigs(show_axes=True, show_unit_circle=True)

x = np.array(range(0,int(500)))
x = x/data.sampling_frequency 
legend = np.array([])

for j,mode in zip(range(len(test_hodmd.modes.T)),test_hodmd.modes.T):
    
    plt.title('Modes')
    plt.plot(mode.real)
    legend = np.append(legend,f"mode {j}")

plt.legend(legend)
    
plt.show()

fig,ax = plt.subplots(nrows = len(test_hodmd.dynamics),ncols = 1,figsize=(5,10))

for j,dynamic in zip(range(len(test_hodmd.dynamics.T)),test_hodmd.dynamics):
    if j == 0:
        ax[j].set_title('Dynamics')
    ax[j].plot(x,dynamic.real)
    ax[j].legend(f"{j}",loc= "upper right")
    #ax[j].set_xlim([0,1])
    
plt.show()

In [ ]:
test_hodmd.original_time['dt'] = test_hodmd.dmd_time['dt'] = x[1] - x[0]
test_hodmd.original_time['t0'] = test_hodmd.dmd_time['t0'] = x[0]
test_hodmd.original_time['tend'] = test_hodmd.dmd_time['tend'] = x[-1]
plt.plot(test_hodmd.original_timesteps, signal_cut, '.', label='snapshots')
plt.plot(test_hodmd.original_timesteps, signal_cut, '-', label='original function')
plt.plot(test_hodmd.dmd_timesteps[:499], test_hodmd.reconstructed_data[0].real, '--', label='DMD output')
plt.legend()
plt.show()

In [ ]:
##Full HODMD method for us

from pydmd import DMD,HODMD
X = np.empty((len(ECG_lead),1000))
for i,j in zip(ECG_lead,range(len(ECG_lead))):
    X[j,:] = dico_ECG[i][:1000]

dmd = HODMD(svd_rank=0, exact=True, opt=True, d=50).fit(X)
for eig in dmd.eigs:
    print('Eigenvalue {}: distance from unit circle {}'.format(eig, np.abs(eig.imag**2+eig.real**2 - 1)))

dmd.plot_eigs(show_axes=True, show_unit_circle=True)

t = np.array(range(1000))
t = t/data.sampling_frequency 
for dynamic in dmd.dynamics:
    plt.plot(t, dynamic.real)
    plt.title('Dynamics')
plt.show()

dmd.original_time['dt'] = dmd.dmd_time['dt'] = t[1] - t[0]
dmd.original_time['t0'] = dmd.dmd_time['t0'] = t[0]
dmd.original_time['tend'] = dmd.dmd_time['tend'] = t[-1]

plt.plot(dmd.original_timesteps, dico_ECG[ECG_lead[9]][:1000], '.', label='snapshots')
plt.plot(dmd.original_timesteps, dico_ECG[ECG_lead[9]][:1000], '-', label='original function')
plt.plot(dmd.dmd_timesteps[:1000-1], dmd.reconstructed_data[9].real, '--', label='DMD output')
plt.legend()
plt.show()

F = np.array([])
for dynamic in dmd.dynamics:
    xa = scipy.signal.hilbert(dynamic.real)
    amplitude_envelope = np.abs(xa)
    instantaneous_phase = np.unwrap(np.angle(xa))
    instantaneous_frequency = (np.diff(instantaneous_phase) /(2.0*np.pi) * data.sampling_frequency)
    F = np.append(F,np.mean(instantaneous_frequency))

print(f"Frequency range : [{np.min(F)};{np.max(F)}] ")
print("Frequencies : ",F)

In [ ]:
D_try = [10,20,30,40,50,60,70,80,90,100,150,200,250,300]

All_lead_MSE = {}
for i,j in zip(ECG_lead,range(len(ECG_lead))):
    MSE = np.array([])
    for d_petit in D_try:
        dmd = HODMD(svd_rank=0, exact=True, opt=True, d=d_petit).fit(X)
        MSE = np.append(MSE,(1/np.mean(X[j,:]))*np.sum((X[j,:]-dmd.reconstructed_data[j].real)**2))
    All_lead_MSE[i] = MSE


for  i in ECG_lead:
    plt.plot(D_try,All_lead_MSE[i],label = f"lead {i.decode('utf8')}") 
plt.legend(loc='best')
plt.xlabel("Values of d")
plt.ylabel("MSE")
plt.grid()
plt.show()


In [ ]:
##Using DMD with data augmentation
from pydmd import DMD,HODMD
X = np.empty((300*6,300))
W = 0

#for k in range(400):
    # if k == X.shape[0]:
    #     break
    # else:
    #     sig_beats = class_beats[ECG_lead[2]][range_beats[ECG_lead[2]][0]]
    #     #sig_beats = dico_ECG[ECG_lead[2]]
    #     X[k,:] = sig_beats[k:400+k]
for k in range(300*6):
    for i,j in zip(ECG_lead,range(len(ECG_lead))):
        if j+W==X.shape[0]:
            break
        else:
            sig_beats = class_beats[i][range_beats[i][0]]
            X[j+W,:] = sig_beats[k:300+k]
        
    W = W+j 
    
dmd = DMD(svd_rank=0, exact=True, opt=True).fit(X)
for eig in dmd.eigs:
    print('Eigenvalue {}: distance from unit circle {}'.format(eig, np.abs(eig.imag**2+eig.real**2 - 1)))

dmd.plot_eigs(show_axes=True, show_unit_circle=True)

t = np.linspace(0,300/data.sampling_frequency,300)
for dynamic in dmd.dynamics:
    plt.plot(t, dynamic.real)
    plt.title('Dynamics')
plt.show()

dmd.original_time['dt'] = dmd.dmd_time['dt'] = t[1] - t[0]
dmd.original_time['t0'] = dmd.dmd_time['t0'] = t[0]
dmd.original_time['tend'] = dmd.dmd_time['tend'] = t[-1]

plt.plot(dmd.original_timesteps[:300], class_beats[ECG_lead[2]][range_beats[ECG_lead[2]][0]][:300], '.', label='snapshots')
plt.plot(dmd.original_timesteps[:300], class_beats[ECG_lead[2]][range_beats[ECG_lead[2]][0]][:300], '-', label='original function')
plt.plot(dmd.dmd_timesteps, dmd.reconstructed_data[2].real, '--', label='DMD output')
plt.legend()
plt.show()

F = np.array([])
for dynamic in dmd.dynamics:
    xa = scipy.signal.hilbert(dynamic.real)
    amplitude_envelope = np.abs(xa)
    instantaneous_phase = np.unwrap(np.angle(xa))
    instantaneous_frequency = (np.diff(instantaneous_phase) /(2.0*np.pi) * data.sampling_frequency)
    F = np.append(F,np.mean(instantaneous_frequency))

print(f"Frequency range : [{np.min(F)};{np.max(F)}] ")
print("Frequencies : ",F)


# dmd.dmd_time["tend"] = 6
# fig = plt.figure(figsize=(15, 5))
# plt.plot(dmd.original_timesteps, class_beats[ECG_lead[8]][range_beats[ECG_lead[8]][0]][:300], '.', label='snapshots')
# plt.plot(np.linspace(0,300/data.sampling_frequency,300), class_beats[ECG_lead[8]][range_beats[ECG_lead[8]][0]][:300], '-', label='original function')
# plt.plot(dmd.dmd_timesteps, dmd.reconstructed_data[0].real, '--', label='DMD output')
# plt.ylim([-1,1])
# plt.legend()
# plt.show()

In [ ]:
norms_eig = np.array([])
print(dmd.eigs.shape)
for eig in dmd.eigs:
    norms_eig = np.append(norms_eig,np.abs(eig.imag**2+eig.real**2))

upper = 1

supper = np.ma.masked_where(norms_eig>upper,norms_eig)
print("Instable eigenvalue : ",len(norms_eig[norms_eig>1]))
smiddle = np.ma.masked_where(norms_eig<1,norms_eig)
print("Stable eigenvalue : ",len(norms_eig[norms_eig<1]))
plt.plot(F,smiddle,".r",F,supper,".b")
plt.xlabel("Frequencies")
plt.ylabel("norms eigenvalues")
#plt.ylim([0.90,1.1])
plt.grid()
plt.show()

In [ ]:
##Applying SVD on the matrix :
X = np.empty((data.signal_length,12))

for i,j in zip(ECG_lead,range(12)):
    X[:,j] = dico_ECG[i]

u,s,vh = np.linalg.svd(X)
print(s)


In [ ]:
###Recreating our own SSA:

dico_ecg_Y = {}
dico_ecg_fc = {}
dico_ecg_sig = {}
def SSA_decomposition(signal_lead,lead,L,fs,name_lead,printing= True):
    #x = signal_lead[:,np.newaxis].copy()
    x = np.c_[signal_lead.copy()].copy()
    y = x.copy()
    N = len(x)
    X = np.zeros([N-L+1,L])
    for k in range(0,N-L+1):
        X[k,:] = np.r_[x[k:k+L,0]]
    
    U,S,V = np.linalg.svd(X,0)
    B = np.zeros(X.shape)
    for i in range(X.shape[0]) :
        for j in range(X.shape[1]) :
            if i == j : 
                B[i,j] = S[j]
    sig = np.diag(B)
    Y = np.zeros([N,L])
    fc = np.zeros([L,1])
    
    for k in range(L):
        Z = np.fliplr(B[k,k]*np.outer(U[:,k],np.r_[V[:,k]]))
        for p in range(-N+L+1,L-1):
            y[L-p] = np.mean(np.diag(Z,p))
        Y[:,k] = y[:,0]
        ##Estimation components frequency:
        f,P = scipy.signal.welch(y[:,0]-np.mean(y[:,0]),fs,noverlap=0,nperseg =np.round(N/4), nfft=4000,return_onesided=True,detrend = False)
        bid = np.max(P)
        Ind = P.argmax()
        fc[k] = f[Ind]
        if not printing:
            plt.figure()
            legend = ["blue : Signal",f"red : component {k}"]
            plt.plot(range(N)/fs,x,range(N),y,"r")
            plt.title(legend)
            plt.xlabel("Seconds")
            plt.legend(f"Frequency : {fc[k]} Hz")
    
    fcs = np.sort(fc,axis = 0)
    index = np.argsort(fc,axis=0)
    sig = np.multiply(sig,sig)


    plt.figure()


    t = np.linspace(0,N/fs,N)    
    fig,ax = plt.subplots(nrows = 1,ncols = 2, figsize = (20,10))
    ax[0].plot(t,signal_lead.copy())
    ax[0].set_title(f"Full signal of Lead {lead.decode('utf8')}")
    ax[0].grid()
    #ax[0].set_xlim([0,3])
    ax[1].set_title(name_lead)
    ax[1].plot(fcs,sig[index],marker = "o")
    ax[1].set_xlabel("Frequencies")
    ax[1].set_xlim([0,80])
    ax[1].set_ylabel("Component Variances")
    ax[1].grid()
    plt.show()

    return Y,fc,sig


for i in ECG_lead:
     Y,fc,sig = SSA_decomposition(dico_ECG[i],i,50,data.sampling_frequency,f"Component variances for lead {i}")
     dico_ecg_Y[i] = Y
     dico_ecg_fc[i] = fc
     dico_ecg_sig[i] = sig

#_,_,_ = SSA_decomposition(dico_ECG[ECG_lead[9]],ECG_lead[9],40,data.sampling_frequency,f"Component variances for lead {ECG_lead[9]}")

In [ ]:
###Let's see recreation of the ECG for each component
##For the p wave:

def wave_reconstructor(signal_lead,lead,fs,Y,fc,sig):

    fcs = np.sort(fc.copy())
    ind = np.argsort(fcs.copy())
    condition_fcs = np.logical_and(fcs>0,fcs<=60)
    condition_fc = np.logical_and(fc>0,fc<=60)
    #reconstruct_signal = np.sum(Y[:,0].copy(),axis = 1)
    reconstruct_signal = Y[:,5]
    t = np.linspace(0,len(signal_lead)/fs,len(signal_lead))
    plt.plot(t,signal_lead,"b",t,reconstruct_signal,"r")
    plt.legend([f"signal of lead {lead.decode('utf8')}","reconstructed wave signal"])
    #plt.xlim([1,3])
    plt.ylim([-signal_lead.max(),signal_lead.max()])
    plt.show()

    
for i in ECG_lead:
    wave_reconstructor(dico_ECG[i],i,data.sampling_frequency,dico_ecg_Y[i],dico_ecg_fc[i],dico_ecg_sig[i])


In [ ]:
##Index SSA

def SSA_index(signal_lead,lead,fs,Y,fc,sig):
    fcs = np.sort(fc.copy())
    ind = np.argsort(fcs.copy())
    P_wave_freq = np.logical_and(fcs>=5,fcs<=30)
    QRS_wave_freq = np.logical_and(fcs>=8,fcs<=50)
    T_wave_freq = np.logical_and(fcs>0,fcs<=10)
    BW_freq = np.logical_and(fcs>=0,fcs<=1)
    N = len(signal_lead)
    Pwave_sigma = np.sum(sig[P_wave_freq[:,0]])/np.sum(sig)
    QRSwave_sigma = np.sum(sig[QRS_wave_freq[:,0]])/np.sum(sig)
    Twave_sigma = np.sum(sig[T_wave_freq[:,0]])/np.sum(sig)
    BW_sigma = np.sum(sig[BW_freq[:,0]])/np.sum(sig)
    PoQRS = Pwave_sigma/QRSwave_sigma
    ToQRS = Twave_sigma/QRSwave_sigma
    RPT = np.abs((Pwave_sigma-Twave_sigma))/Pwave_sigma
    t = np.linspace(0,N/fs,N) 
    plt.figure() 
    plt.plot(t,signal_lead.copy())
    plt.title(f"Full signal of Lead {lead.decode('utf8')}")
    plt.grid()
    #plt.xlim([0,3])
    plt.figtext(1, 0.7, "Relative importance P_wave = {0:.2f}".format(Pwave_sigma))
    plt.figtext( 1, 0.6, "Relative importance T_wave = {0:.2f}".format(Twave_sigma))
    plt.figtext( 1, 0.5, "Relative importance QRS_wave = {0:.2f}".format(QRSwave_sigma))
    plt.figtext( 1, 0.4, "Relative importance BW = {0:.2f}".format(BW_sigma))
    plt.figtext( 1, 0.3, "Pwave/QRSwave = {0:.2f}".format(PoQRS))
    plt.figtext( 1, 0.2, "Twave/QRSwave = {0:.2f}".format(ToQRS))
    plt.figtext( 1, 0.1, "Relative diff P and T wave = {0:.2f}".format(RPT))


# for i in ECG_lead:
#     SSA_index(dico_ECG[i],i,data.sampling_frequency,dico_ecg_Y[i],dico_ecg_fc[i],dico_ecg_sig[i]) 



In [ ]:
### Generalized Hurst method : Taken from https://github.com/PTRRupprecht/GenHurst/blob/master/genhurst.py

import warnings
 
def genhurst(S,q):

    L=len(S)       
    if L < 100:
        warnings.warn('Data series very short!')
       
    H = np.zeros((len(range(5,20)),1))
    k = 0
    
    for Tmax in range(5,20):
        
        x = np.arange(1,Tmax+1,1)
        mcord = np.zeros((Tmax,1))
        
        for tt in range(1,Tmax+1):
            dV = S[np.arange(tt,L,tt)] - S[np.arange(tt,L,tt)-tt] 
            VV = S[np.arange(tt,L+tt,tt)-tt]
            N = len(dV) + 1
            X = np.arange(1,N+1,dtype=np.float64)
            Y = VV
            mx = np.sum(X)/N
            SSxx = np.sum(X**2) - N*mx**2
            my = np.sum(Y)/N
            SSxy = np.sum( np.multiply(X,Y))  - N*mx*my
            cc1 = SSxy/SSxx
            cc2 = my - cc1*mx
            ddVd = dV - cc1
            VVVd = VV - np.multiply(cc1,np.arange(1,N+1,dtype=np.float64)) - cc2
            mcord[tt-1] = np.mean( np.abs(ddVd)**q )/np.mean( np.abs(VVVd)**q )
            
        mx = np.mean(np.log10(x))
        SSxx = np.sum( np.log10(x)**2) - Tmax*mx**2
        my = np.mean(np.log10(mcord))
        SSxy = np.sum( np.multiply(np.log10(x),np.transpose(np.log10(mcord)))) - Tmax*mx*my
        H[k] = SSxy/SSxx
        k = k + 1
        
    mH = np.mean(H)/q
    
    return mH

In [ ]:
####Hurst exponents test : 


fs = data.sampling_frequency
x = np.array(range(0,len(dico_ECG[ECG_lead[0]])))
x = x/fs 
segment_length = 100

for i in ECG_lead:
    sig = dico_ECG[i]
    H = genhurst(sig,1)
    print(f"Hurst exponent val for lead {i.decode('utf8')} : ", H)
    print(f"Fractal dimension for lead {i.decode('utf8')} : ", 2-H)
    plt.plot(x,sig)
    plt.title(f"lead {i.decode('utf8')}")
    plt.xlabel("Time (sec)")
    plt.ylabel("Voltage amplitude")
    plt.xlim([0,3])
    plt.grid()
    plt.show()
    # H_arr = np.array([])
    # w = 1
    # while (w + segment_length) <= len(dico_ECG[i]):
    #     sig_c = dico_ECG[i][int((w - 1)) : int((w) + segment_length)]
    #     H= genhurst(sig_c,1)
    #     H_arr = np.append(H_arr,H)
    #     w+=1
    # interval = range(1,w)
    # f, ax = plt.subplots(nrows=2,ncols = 1,figsize = (20,10))
    # ax[0].plot(interval/fs,H_arr)
    # ax[0].set_title(f"Fractale for lead {i.decode('utf8')} for an interval of length {segment_length}")
    # ax[0].set_xlabel("interval sequence")
    # ax[0].set_ylabel("Hurst exponent")
    # ax[0].grid()
    # ax[1].plot(x,dico_ECG[i])
    # ax[1].set_title(f"signal of lead {i.decode('utf8')}")
    # ax[1].set_xlabel("Time")
    # ax[1].set_ylabel("Voltage amplitude")
    # ax[1].grid()
    # plt.show()

    #print("H={:.4f}, c={:.4f}".format(H,c))
    #print("D = {:.4f}".format(2-H))

##For a constant signal:
# sig_const = np.ones([5000])
# print("H = ",genhurst(sig_const,1))
# print("D = ",2-genhurst(sig_const,1))

In [ ]:
##HIGUCHI method on all the signal
import os
sys.path.append(os.path.join(os.getcwd(), ".."))

from shared_utils import Time_series_dimensions_calculus as TSD

for i in ECG_lead:
    L1 = TSD.Lq_k(dico_ECG[i][:2400],1,fs)
    L2 = TSD.Lq_k(dico_ECG[i][:2400],2,fs)
    D = (np.log(L1)-np.log(L2))/np.log(2)
    print(f"For lead {i.decode('utf8')} : ",D)

In [ ]:
###Create Correlation matrix
print([i.decode('utf8') for i in ECG_lead])
ECG_frame = pd.DataFrame(dico_ECG,columns = ECG_lead)
matrix = ECG_frame.corr()
print(matrix)

In [ ]:
###Definition SNR observational from artice ECGAccess : 

def SNRObs(signal,fs):
    f,PSD = periodogram(signal,fs)
    Sig_PSD = np.sum(PSD[np.logical_and(f>2,f<=40)])
    LF_PSD = np.sum(PSD[np.logical_and(f>=0,f<=2)])
    HF_PSD = np.sum(PSD[np.logical_and(f>40,f<=250)])
    return Sig_PSD/(LF_PSD+HF_PSD)


In [ ]:
for i in ECG_lead:
    print(f"For lead {i.decode('utf8')} : SNR =", SNRObs(dico_ECG[i],fs))
    print(f"For lead {i.decode('utf8')} : SNR (db) =", 10*np.log10(SNRObs(dico_ECG[i],fs)))

In [ ]:
##Study : TSD vs SNR for ECG (no lead differentiation):

Big_dico_SNR = {}
Big_dico_TSD = {}

for n in ECG_lead:
    Big_dico_SNR[n] = np.array([])
    Big_dico_TSD[n] = np.array([])

with make_reader(path_petastorm) as reader:
    for sample in reader:
        data = sample
        ECG_signal = data.signal
        ECG_lead = data.signal_names
        fs = data.sampling_frequency
        dico_ECG = {}

        for i,j in zip(ECG_lead,range(12)):
            dico_ECG[i] = ECG_signal[:,j]

        for name in ECG_lead:
            Big_dico_SNR[name] = np.append(Big_dico_SNR[name],10*np.log10(SNRObs(dico_ECG[name],fs)))
            L1 = TSD.Lq_k(dico_ECG[name],1,fs)
            L2 = TSD.Lq_k(dico_ECG[name],2,fs)
            D = (np.log(L1)-np.log(L2))/np.log(2)
            Big_dico_TSD[name] = np.append(Big_dico_TSD[name],D)


for n in ECG_lead:
    plt.figure()
    Big_dico_TSD[n] = Big_dico_TSD[n][np.argwhere(~np.isnan(Big_dico_SNR[n]))]
    Big_dico_SNR[n] = Big_dico_SNR[n][~np.isnan(Big_dico_SNR[n])]
    SNR_val_ord = np.sort(Big_dico_SNR[n].copy())
    Ind_ord = np.argsort(Big_dico_SNR[n].copy())
    plt.plot(SNR_val_ord,Big_dico_TSD[n][Ind_ord].copy(),"ob",label = n.decode('utf8'))
    plt.legend(loc="best", bbox_to_anchor=(1, 1))
    plt.xlabel("SNR (dB)")
    plt.ylabel("TSD Value")
    plt.xscale("log")
    plt.title("TSD ECG vs SNR observational")
    plt.grid()
    plt.show()
    
        

In [ ]:
##Flatline detector : 

def flatline_sig (sig,t):
    series = sig.copy()
    index = np.where(np.diff(series)!=0.0,np.nan,True)
    index = np.append(index,np.nan)
    copy_sig = sig.copy()
    return copy_sig[index==True]

def is_flatline(sig):
    cond = np.where(np.diff(sig.copy())!=0.0,np.nan,True)
    if (np.isnan(cond).any()):
        return False
    else : 
        return True

def is_segment_flatline(sig):
    cond = np.where(np.diff(sig.copy())!=0.0,np.nan,True)
    if (np.isnan(cond).any() or len(cond[cond==True])<0.75*len(sig)):
        return False
    else : 
        return True

##Flatline signal
c = np.ones([5000])
t = np.linspace(0,int(len(c)/500),len(c))

print("Is a Constant signal a flatline (Answer : YES)? Machine Says : ", is_segment_flatline(c))

for i in ECG_lead:
    t = np.linspace(0,int(len(dico_ECG[i])/fs),len(dico_ECG[i]))
    _,const_rpeaks = nk.ecg_peaks(dico_ECG[i],sampling_rate=500)
    print(const_rpeaks)
    print(f"Is Lead {i.decode('utf8')} a flatline ? Machine Says : ",is_segment_flatline(dico_ECG[i]))

In [ ]:
####Peak detections and morphological P wave extraction : 
from math import isnan
import biosppy
import neurokit2 as nk
from shared_utils.Time_series_dimensions_calculus import TSD_index_lead

def PQRST_template_extractor(ECG_signal,rpeaks,before  = 200,after = 400):
    ##From the Biosspy function _extract_heartbeats
    R = np.sort(rpeaks)
    length = len(ECG_signal)
    templates = []
    newR = []

    for r in R:
        a = r - before
        if a < 0:
            continue
        b = r + after
        if b > length:
            break
        templates.append(ECG_signal[a:b])
        newR.append(r)

    templates = np.array(templates)
    newR = np.array(newR, dtype="int")

    return templates, newR

big_mean_rp = np.array([])
detect = Detectors(fs)
for i in ECG_lead:
    dico_HB = nk.ecg_segment(dico_ECG[i],sampling_rate = fs,show = True)
    # rpeaks = detect.pan_tompkins_detector(dico_ECG[i])
    # qrs_epochs,_ = biosppy.signals.ecg.extract_heartbeats(dico_ECG[i],rpeaks,sampling_rate = fs)
#print(qrs_epochs)
    plt.figure()
    plt.plot(np.linspace(0,int(len(dico_ECG[i])/fs),len(dico_ECG[i])),dico_ECG[i])
    plt.show()
    plt.figure()
    # for qrs in qrs_epochs:
    #     plt.plot(qrs)
    qrs_epochs = np.array([dico_HB[j]["Signal"].to_numpy() for j in list(dico_HB.keys())])
    sig_mean = qrs_epochs[0]
    for w in range(1,len(qrs_epochs)):
        sig_mean += qrs_epochs[w]


    sig = sig_mean/len(qrs_epochs)
    plt.figure()
    plt.plot(sig)
    plt.show()
    r_p = np.array([])
    for j in range(len(qrs_epochs)):
        beats = qrs_epochs[j]
        print(pearsonr(sig,beats))
        r_p = np.append(r_p,pearsonr(sig,beats)[0])
        print(np.corrcoef(sig,beats))

    print("R Pearson QRS complex : ",np.mean(r_p))
    big_mean_rp = np.append(big_mean_rp,np.mean(r_p))

print("OVERALL : ",np.mean(big_mean_rp))

##Pearson R coefficient:
# r_p = np.array([])
# for w in range(int(len(dico_ECG[ECG_lead[ind]])-len(sig))):
#     signal_test = dico_ECG[ECG_lead[ind]][w:w+len(sig)]
#     r = pearsonr(signal_test,sig)
#     r_p = np.append(r_p,r)
# r_p = np.correlate(sig,dico_ECG[ECG_lead[ind]])
# plt.figure()
# plt.plot(np.linspace(0,len(r_p),len(r_p))/fs,r_p)
# plt.show()
# print("Mean R coeff : ",np.mean(r_p))

# _,rpeaks = nk.ecg_peaks(dico_ECG[ECG_lead[ind]],sampling_rate=500)
# _, waves_peak = nk.ecg_delineate(dico_ECG[ECG_lead[ind]], 
#                              rpeaks, 
#                              sampling_rate=500, 
#                              method="cwt", 
#                             )
# print(waves_peak)

# # plot = nk.events_plot([waves_peak['ECG_T_Peaks'], 
# #                    waves_peak['ECG_P_Peaks'],
# #                    waves_peak['ECG_Q_Peaks'],
# #                    waves_peak['ECG_S_Peaks']], dico_ECG[i])
    

In [ ]:
from biosppy.signals import ecg

sig = dico_ECG[ECG_lead[0]]

ecg_test = ecg.ecg(sig,fs,show = False)

r_peaks = ecg_test[2]
templates = ecg_test[4]
templates_t =ecg_test[3]
for i in templates:
    plt.plot(templates_t,i)
    
HR = ecg_test[6]
print(r_peaks)
print(templates)
    
print(HR)

In [ ]:
##Trial cross correlation : 
#sig_ref = dico_ECG[ECG_lead[0]]
for i in ECG_lead:
    corr = correlate(dico_ECG[i], dico_ECG[i])
    lags = correlation_lags(len(dico_ECG[i]), len(dico_ECG[i]))
    print(len(lags))
    corr /= np.max(corr)
    print(np.max(corr))
    print(np.min(corr))
    print(np.mean(np.abs(corr)))
    fig, (ax_orig, ax_noise, ax_corr) = plt.subplots(3, 1, figsize=(4.8, 4.8))
    ax_orig.plot(dico_ECG[i])
    ax_orig.set_title(f"Lead {ECG_lead[0].decode('utf8')}")
    ax_orig.set_xlabel('Sample Number')
    ax_noise.plot(dico_ECG[i])
    ax_noise.set_title(f"Lead {i.decode('utf8')}")
    ax_noise.set_xlabel('Sample Number')
    ax_corr.plot(lags, corr)
    ax_corr.set_title('Cross-correlated signal')
    ax_corr.set_xlabel('Lag')
    ax_orig.margins(0, 0.1)
    ax_noise.margins(0, 0.1)
    ax_corr.margins(0, 0.1)
    fig.tight_layout()
    plt.show()

In [ ]:
###correlation matrix:
dico_segment = {}

for i in ECG_lead :
    plt.figure() 
    dico_HB = nk.ecg_segment(dico_ECG[i],sampling_rate = fs)
    template = np.array([dico_HB[d]["Signal"].to_numpy() for d in list(dico_HB.keys())])
    
    dico_segment[i] = template

X_data = np.empty([12,dico_segment[ECG_lead[0]].shape[0],dico_segment[ECG_lead[0]].shape[1]])

for j in range(len(ECG_lead)) : 
    X_data[j,:,:] = dico_segment[ECG_lead[j]]

X_self_corr = np.empty([dico_segment[ECG_lead[0]].shape[0],12])
for w in range(X_data.shape[1]):
    X = X_data[:,w,:]
    for sig in range(X.shape[0]):
        corr = correlate(X[sig,:], X[sig,:])
        lags = correlation_lags(len(X[sig,:]), len(X[sig,:]))

        # fig, (ax_orig, ax_noise, ax_corr) = plt.subplots(3, 1, figsize=(4.8, 4.8))
        # ax_orig.plot(X[sig,:])
        # ax_orig.set_title(f"Lead {ECG_lead[sig].decode('utf8')}")
        # ax_orig.set_xlabel('Sample Number')
        # ax_noise.plot(X[sig,:])
        # ax_noise.set_title(f"Lead {ECG_lead[sig].decode('utf8')}")
        # ax_noise.set_xlabel('Sample Number')
        # ax_corr.plot(lags, corr)
        # ax_corr.set_title('Cross-correlated signal')
        # ax_corr.set_xlabel('Lag')
        # ax_orig.margins(0, 0.1)
        # ax_noise.margins(0, 0.1)
        # ax_corr.margins(0, 0.1)
        # fig.tight_layout()
        # plt.show()
        # print(np.max(corr))
    # cross_cor_mat = np.corrcoef(X)
   
    # #matrix_int = np.triu(cross_cor_mat,1)
    # np.set_printoptions(precision=2,suppress=True)
    # for j in range(len(ECG_lead)):
    #     print(f"Mean correlation coefficient for lead {ECG_lead[j].decode('utf8')} : ", np.mean(np.abs(cross_cor_mat[j,:]))-1/12)

In [ ]:
##Cross correlation matrix : 

Cross_corr = np.empty([12,12])
coordinates = [(x,y) for x in range(12) for y in range(12)]
for c in coordinates : 
    Cross_corr[c[0],c[1]] = np.mean(np.matmul(dico_ECG[ECG_lead[c[0]]],dico_ECG[ECG_lead[c[1]]]))
np.set_printoptions(precision=1,suppress=True)
print(Cross_corr)

In [ ]:
##overlapping lead : 
def overlap_percentage(xlist,ylist):
    min1 = np.min(xlist)
    max1 = np.max(xlist)
    min2 = np.min(ylist)
    max2 = np.max(ylist)

    overlap = np.maximum(0, np.minimum(max1, max2) - np.maximum(min1, min2))
    length = max1-min1 + max2-min2
    lengthx = max1-min1
    lengthy = max2-min2

    return 2*overlap/length , overlap/lengthx  , overlap/lengthy 

def overlapping_sig(dico_signal,name_lead):

    X = np.empty([12,len(dico_signal[name_lead[0]])])

    s = 5

    for i in range(len(name_lead)):
        X[i,:] = dico_signal[name_lead[i]].copy()+s*i
    
    ALL_THE_OVERLAP_OF_NIGHTMARE = np.zeros([len(name_lead),len(name_lead)])
    for j in range(len(name_lead)):
        sig = X[j,:]
        list_travel = list(range(len(name_lead)))
        list_travel.remove(j)
        for i in list_travel:
            p,_,_ = overlap_percentage(sig,X[i,:])
            if p!=0:
                ALL_THE_OVERLAP_OF_NIGHTMARE[j,i] += p
                

    return ALL_THE_OVERLAP_OF_NIGHTMARE


def overlapping_sig_plot(dico_signal,name_lead,fs):
    X = np.empty([12,len(dico_signal[name_lead[0]])])
    s = 5
    for j in range(len(name_lead)):
        X[j,:] = dico_signal[name_lead[j]].copy()
    
    t = np.linspace(0,int(len(X[0,:])/fs),len(X[0,:]))

    fig,ax = plt.subplots()
    
    for i in range(X.shape[0]):
        ax.plot(t,X[i,:]+s*i)

    labels = [f"{i.decode('utf8')}" for i in name_lead]
    ax.set_yticks(np.arange(0,X.shape[0])*s)
    ax.set_yticklabels(labels)
    for t, l in zip(ax.get_yticklabels(), ax.lines):
        t.set_color(l.get_color())
    plt.grid()
    plt.show()

O = overlapping_sig(dico_ECG,ECG_lead) 
np.set_printoptions(precision=2,suppress=True)
print(O) 


In [ ]:
def Missing_lead_score(dico_signal,name_lead):
    H_thres_ECG = 1
    L_thres_ECG = -1
    X = np.empty([12,len(dico_signal[name_lead[0]])])
    for j in range(len(name_lead)):
        X[j,:] = dico_signal[name_lead[j]].copy()

    F = np.zeros([len(name_lead),len(name_lead)])
    for e in range(len(name_lead)) :
        sig = X[e,:].copy()
        if H_thres_ECG-np.max(sig) >=0 and L_thres_ECG-np.min(sig)>0:
            p = np.abs((L_thres_ECG-np.min(sig))/np.min(sig))
        elif H_thres_ECG-np.max(sig) <0 and L_thres_ECG-np.min(sig)<=0:
            p = np.abs((H_thres_ECG-np.max(sig))/np.max(sig))
        elif H_thres_ECG-np.max(sig) <0 and L_thres_ECG-np.min(sig)>0:
            p = ((np.abs((H_thres_ECG-np.max(sig))/np.max(sig))+np.abs((L_thres_ECG-np.min(sig))/np.min(sig))))/2
            
        else : 
            p = 0
        
        F[e,e] += p
        if e!=len(name_lead)-1:
            F[e,e+1:] += p/11
        if e>0:
            F[e,:e]+=p/11


    return F

P = Missing_lead_score(dico_ECG,ECG_lead)
np.set_printoptions(precision=2,suppress=True)
print(P)

In [ ]:
def SDR_matrix(dico_signal,name_lead,fs):
    
    X = np.empty([12,len(dico_signal[name_lead[0]])])
    for f in range(len(name_lead)):
        X[f,:] = dico_signal[name_lead[f]].copy()
    S = np.zeros([12,12])
    for j in range(X.shape[0]):
        f,PSD = periodogram(X[j,:],fs)
        QRS_signal_PSD = np.sum(PSD[np.logical_and(f>=5,f<=14)])
        ECG_tot = np.sum(PSD[np.logical_and(f>=5,f<=50)],dtype = np.float64)
        if ECG_tot == 0:
            ECG_tot = np.sum(np.abs(X[j,:])**2)
            if ECG_tot ==0:
                ECG_tot = 2**63-1
        SDR_val = QRS_signal_PSD/ECG_tot
        rho = max([0.5-SDR_val,SDR_val-0.8,0])
        S[j,j] += rho
        list_travel = list(range(len(name_lead)))
        list_travel.remove(j)
        for i in list_travel:
            S[j,i] += rho/11
    return S

SDR = SDR_matrix(dico_ECG,ECG_lead,fs)
np.set_printoptions(precision=2,suppress=True)
print(SDR)

In [ ]:
def Global_score(SM,FM,OM,SDM):
    M = SM+FM+OM+SDM
    print(M)
    # results = np.array([])
    # for j in range(M.shape[0]):
    #     results = np.append(results,np.sum(M[:,j]))
    results = np.max(np.abs(np.linalg.eigvals(M)))
    return results

def flatline_score(signal):
    cond = np.where(np.diff(signal.copy())!=0.0,np.nan,True)
    if (len(cond[cond==True])<0.20*len(signal)):
        return len(cond[cond==True])/len(signal)
    else : 
        return 0


def Flatline_matrix(dico_signal,name_lead):
    X = np.empty([12,len(dico_signal[name_lead[0]])])
    for f in range(len(name_lead)):
        X[f,:] = dico_signal[name_lead[f]].copy()
    
    F = np.zeros([len(name_lead),len(name_lead)])
    for j in range(len(name_lead)):
        p = flatline_score(X[j,:])
        if p!=0:
            F[j,j] += p
            if j!=len(name_lead)-1:
                F[j,j+1:] += p/11
                if j>0:
                    F[j,:j-1]+=p/11
    return F

FM = Flatline_matrix(dico_ECG,ECG_lead)
np.set_printoptions(precision=3,suppress=True)
print(Global_score(SDR,P,O,FM))

In [ ]:
detect = Detectors(fs)
rpeaks = detect.pan_tompkins_detector(dico_ECG[ECG_lead[0]])
qrs_epochs,_ = biosppy.signals.ecg.extract_heartbeats(dico_ECG[ECG_lead[0]],rpeaks,sampling_rate = fs)
print(qrs_epochs.shape)

In [ ]:

def is_segment_flatline(sig):
    cond = np.where(np.diff(sig.copy())!=0.0,np.nan,True)
    if (len(cond[cond==True])<0.40*len(sig)):
        return False
    else : 
        return True

def PQRST_template_extractor(ECG_signal,rpeaks,before  = 200,after = 400):
    ##From the Biosspy function _extract_heartbeats
    R = np.sort(rpeaks)
    length = len(ECG_signal)
    templates = []
    newR = []

    for r in R:
        a = r - before
        if a < 0:
            continue
        b = r + after
        if b > length:
            break
        templates.append(ECG_signal[int(r-np.median(np.diff(rpeaks))/2):int(r+np.median(np.diff(rpeaks))/2)])
        newR.append(r)

    templates = np.array(templates)
    newR = np.array(newR, dtype="int")

    return templates, newR

def Morph_score(signals,name_lead,fs):
    QRS_lead = {}
    QRS_templates = {}
    QRS_arr = np.array([])
    detect = Detectors(fs)
    for i in name_lead:
        r_peaks = detect.pan_tompkins_detector(dico_ECG[i])
        if is_segment_flatline(signals[i]) or len(r_peaks)<=2:
            QRS_lead[i] = (0,signals[i])
            QRS_arr = np.append(QRS_arr,0)
            pass
        else :
           
            template,_ =PQRST_template_extractor(dico_ECG[i],rpeaks = r_peaks)
            index_maxima = [np.argmax(template[w,:]) for w in range(template.shape[0])]
            median_index = np.median(index_maxima)
            templates_good = template[np.isclose(index_maxima,median_index,rtol=0.1)]
            #print(np.median([np.argmax(template[w,:]) for w in range(template.shape[0])]))
            # plt.figure()
            # QRS_epochs = nk.ecg_segment(signals[i],sampling_rate = fs,show = True)
            # template = np.array([QRS_epochs[d]["Signal"].to_numpy() for d in list(QRS_epochs.keys())])
            
            QRS_templates[i] = templates_good
            sig_mean = templates_good[0].copy()
            for j in range(1,templates_good.shape[0]):
                sig_mean +=templates_good[j,:].copy()
            
            sig = sig_mean/templates_good.shape[0]
            r_p = np.array([])
            for w in range(templates_good.shape[0]):
                beats = templates_good[w,:].copy()
                r_p = np.append(r_p,pearsonr(sig,beats)[0])
            QRS_lead[i] = (np.mean(r_p.copy()),signals[i])
            QRS_arr = np.append(QRS_arr,np.mean(r_p.copy()))
    return QRS_lead,QRS_templates

In [ ]:
##Trial : 
QRS_lead,QRS_temp = Morph_score(dico_ECG,ECG_lead,fs)
for n in ECG_lead:
    print("Mean Correlation coefficient value : ",QRS_lead[n][0])
    plt.figure()
    plt.plot(QRS_lead[n][1])
    plt.title(f"Lead {n.decode('utf8')}")
    plt.show()
    plt.figure()
    
    for j in QRS_temp[n]:
        plt.title(f"QRS complex segment for lead {n.decode('utf8')}")
        plt.plot(j)
    plt.show()

In [ ]:
##Trial TSD : 
t = [1,100,500,1000,2000,3000]
TST = np.empty([len(t),12])
for k in range(len(t)):
    dic_test_TSD,val_mean = TSD.TSD_index(dico_ECG,ECG_lead,fs,t[k])
    TST[k,:] = np.array([dic_test_TSD[j][0] for j in ECG_lead])
labels = []
plt.figure()
for k in range(len(t)):   
    plt.plot(ECG_lead,TST[k,:])
    labels.append(f"k = {t[k]}")
plt.legend(labels)
plt.title("TSD mean calculation for each mean with different step")
#plt.xlim([0,1])
plt.grid()
plt.show()


In [ ]:
###Test TSD index
truc,much = TSD.TSD_index(dico_ECG,ECG_lead,fs)
print(much)